## Interactive Brokers Trading Bot

An Interactive Brokers trading bot supporting both paper trading (testing) and live trading without deep reinforcement learning.

### Import Dependencies and Setup

In [17]:
# Import required libraries
import asyncio
import logging
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Union
import time
import random

# Interactive Brokers imports
from ib_insync import IB, Stock, MarketOrder, LimitOrder, Contract, Ticker
from ib_insync import util

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('trading_bot.log')
    ]
)

# Suppress ib_insync debug logs
logging.getLogger('ib_insync').setLevel(logging.WARNING)

print("✓ Dependencies imported successfully")
print("✓ Logging configured")

✓ Dependencies imported successfully
✓ Logging configured


### IB Environment Configuration

In [18]:
class IBEnvironment:
    """Configuration class for IB connection environments"""
    
    # Paper Trading Configuration
    PAPER_TRADING = {
        'host': '127.0.0.1',
        'port': 7497,  # Paper trading port
        'client_id': 1,
        'description': 'Paper Trading (Demo Account)'
    }
    
    # Live Trading Configuration
    LIVE_TRADING = {
        'host': '127.0.0.1',
        'port': 7496,  # Live trading port
        'client_id': 2,
        'description': 'Live Trading (Real Money)'
    }
    
    @classmethod
    def get_config(cls, environment: str = 'paper') -> Dict:
        """
        Get configuration for specified environment
        
        Args:
            environment: 'paper' for demo trading, 'live' for real money
        """
        if environment.lower() == 'paper':
            return cls.PAPER_TRADING.copy()
        elif environment.lower() == 'live':
            return cls.LIVE_TRADING.copy()
        else:
            raise ValueError("Environment must be 'paper' or 'live'")
    
    @classmethod
    def list_environments(cls):
        """List available trading environments"""
        print("Available Trading Environments:")
        print(f"  paper: {cls.PAPER_TRADING['description']} - Port {cls.PAPER_TRADING['port']}")
        print(f"  live:  {cls.LIVE_TRADING['description']} - Port {cls.LIVE_TRADING['port']}")

# Test the environment configuration
IBEnvironment.list_environments()
print("\n✓ IB Environment configuration ready")

Available Trading Environments:
  paper: Paper Trading (Demo Account) - Port 7497
  live:  Live Trading (Real Money) - Port 7496

✓ IB Environment configuration ready


### IB Connection Class

In [19]:
class IBConnection:
    """Interactive Brokers connection handler"""
    
    def __init__(self, environment: str = 'paper'):
        """
        Initialise IB connection
        
        Args:
            environment: 'paper' for demo, 'live' for real money
        """
        self.config = IBEnvironment.get_config(environment)
        self.environment = environment
        self.ib = IB()
        self.connected = False
        self.logger = logging.getLogger(f'IBConnection-{environment}')
        
        # Connection parameters
        self.host = self.config['host']
        self.port = self.config['port']
        self.client_id = self.config['client_id']
        
        self.logger.info(f"Initialised for {self.config['description']}")
    
    async def connect(self, timeout: int = 10) -> bool:
        """Connect to Interactive Brokers TWS or Gateway"""
        try:
            self.logger.info(f"Connecting to {self.config['description']}...")
            
            await self.ib.connectAsync(
                host=self.host,
                port=self.port,
                clientId=self.client_id,
                timeout=timeout
            )
            
            self.connected = True
            self.logger.info(f"✓ Connected to IB on port {self.port}")
            
            # Get account information
            account = self.ib.managedAccounts()[0] if self.ib.managedAccounts() else "Unknown"
            self.logger.info(f"Account: {account}")
            
            return True
            
        except Exception as e:
            self.logger.error(f"Failed to connect: {e}")
            self.logger.error("Ensure TWS or IB Gateway is running and accepting connections")
            self.connected = False
            return False
    
    def connect_sync(self, timeout: int = 10) -> bool:
        """Synchronous connection method - Jupyter notebook compatible"""
        try:
            # Check if we're in a Jupyter notebook with existing event loop
            try:
                import IPython
                if IPython.get_ipython() is not None:
                    # We're in Jupyter - use util.startLoop for compatibility
                    util.startLoop()
            except ImportError:
                pass  # Not in Jupyter
            
            self.ib.connect(
                host=self.host,
                port=self.port,
                clientId=self.client_id,
                timeout=timeout
            )
            
            self.connected = True
            self.logger.info(f"✓ Connected to IB on port {self.port}")
            
            # Get account information
            try:
                account = self.ib.managedAccounts()[0] if self.ib.managedAccounts() else "Unknown"
                self.logger.info(f"Account: {account}")
            except:
                pass  # Account info not critical
            
            return True
            
        except ConnectionRefusedError as e:
            self.logger.error(f"API connection failed: {e}")
            self.logger.error("Make sure API port on TWS/IBG is open")
            self.connected = False
            return False
        except Exception as e:
            if "This event loop is already running" in str(e):
                self.logger.error("Event loop conflict - trying alternative connection method...")
                return self._connect_alternative(timeout)
            else:
                self.logger.error(f"Failed to connect: {e}")
                self.connected = False
                return False
    
    def _connect_alternative(self, timeout: int = 10) -> bool:
        """Alternative connection method for Jupyter notebooks"""
        try:
            # Create a new IB instance to avoid event loop conflicts
            self.ib = IB()
            
            # Try connecting without event loop management
            self.ib.connect(
                host=self.host,
                port=self.port,
                clientId=self.client_id,
                timeout=timeout
            )
            
            self.connected = True
            self.logger.info(f"✓ Connected to IB on port {self.port} (alternative method)")
            
            return True
            
        except ConnectionRefusedError as e:
            self.logger.error(f"API connection failed: {e}")
            self.logger.error("Make sure TWS or IB Gateway is running and API is enabled")
            self.connected = False
            return False
        except Exception as e:
            self.logger.error(f"Alternative connection failed: {e}")
            self.connected = False
            return False
    
    def disconnect(self):
        """Disconnect from Interactive Brokers"""
        try:
            if self.connected and self.ib.isConnected():
                self.ib.disconnect()
                self.logger.info("Disconnected from IB")
        except Exception as e:
            self.logger.error(f"Error during disconnect: {e}")
        finally:
            self.connected = False
    
    def is_connected(self) -> bool:
        """Check if connection is active"""
        return self.connected and self.ib.isConnected()
    
    def get_account_summary(self) -> Dict:
        """Get account summary information"""
        if not self.is_connected():
            self.logger.warning("Not connected to IB")
            return {}
        
        try:
            account = self.ib.managedAccounts()[0]
            summary = self.ib.accountSummary(account)
            
            # Convert to dictionary
            account_info = {}
            for item in summary:
                account_info[item.tag] = item.value
            
            return account_info
            
        except Exception as e:
            self.logger.error(f"Error getting account summary: {e}")
            return {}

print("✓ IB Connection class created")

✓ IB Connection class created


### Historical Data Handler

In [20]:
class HistoricalDataHandler:
    """Handle historical data retrieval from Interactive Brokers"""
    
    def __init__(self, ib_connection: IBConnection):
        """
        Initialise historical data handler
        
        Args:
            ib_connection: IBConnection instance
        """
        self.ib_conn = ib_connection
        self.ib = ib_connection.ib
        self.logger = logging.getLogger('HistoricalDataHandler')
        self.data_cache = {}
        
    def create_contract(self, symbol: str, exchange: str = 'SMART', 
                       currency: str = 'USD') -> Stock:
        """
        Create stock contract for IB
        
        Args:
            symbol: Stock symbol (e.g., 'MRK')
            exchange: Exchange (default: 'SMART')
            currency: Currency (default: 'USD')
        """
        return Stock(symbol, exchange, currency)
    
    def get_historical_data(self, symbol: str, duration: str = '1 D',
                           bar_size: str = '1 min', exchange: str = 'SMART',
                           currency: str = 'USD') -> Optional[pd.DataFrame]:
        """
        Get historical data for analysis
        
        Args:
            symbol: Stock symbol
            duration: Data duration (e.g., '1 D', '1 W', '1 M', '1 Y')
            bar_size: Bar size (e.g., '1 min', '5 mins', '1 hour', '1 day')
            exchange: Exchange (default: 'SMART')
            currency: Currency (default: 'USD')
        """
        if not self.ib_conn.is_connected():
            self.logger.error("Not connected to IB")
            return None
        
        cache_key = f"{symbol}_{duration}_{bar_size}"
        
        try:
            contract = self.create_contract(symbol, exchange, currency)
            
            bars = self.ib.reqHistoricalData(
                contract,
                endDateTime='',
                durationStr=duration,
                barSizeSetting=bar_size,
                whatToShow='TRADES',
                useRTH=True,
                formatDate=1
            )
            
            if bars:
                df = util.df(bars)
                df['timestamp'] = pd.to_datetime(df['date'])
                df['symbol'] = symbol
                
                # Cache the data
                self.data_cache[cache_key] = {
                    'data': df,
                    'retrieved_at': datetime.now()
                }
                
                self.logger.info(f"Retrieved {len(df)} historical bars for {symbol}")
                return df
            else:
                self.logger.warning(f"No historical data for {symbol}")
                return None
                
        except Exception as e:
            self.logger.error(f"Error getting historical data for {symbol}: {e}")
            return None
    
    def get_intraday_data(self, symbol: str, days: int = 1) -> Optional[pd.DataFrame]:
        """
        Get intraday historical data (1-minute bars)
        
        Args:
            symbol: Stock symbol
            days: Number of days (max 30 for 1-min bars)
        """
        duration = f"{days} D"
        return self.get_historical_data(symbol, duration, '1 min')
    
    def get_daily_data(self, symbol: str, period: str = '1 M') -> Optional[pd.DataFrame]:
        """
        Get daily historical data
        
        Args:
            symbol: Stock symbol
            period: Time period (e.g., '1 M', '3 M', '6 M', '1 Y', '2 Y')
        """
        return self.get_historical_data(symbol, period, '1 day')
    
    def get_weekly_data(self, symbol: str, period: str = '1 Y') -> Optional[pd.DataFrame]:
        """
        Get weekly historical data
        
        Args:
            symbol: Stock symbol
            period: Time period (e.g., '1 Y', '2 Y', '5 Y')
        """
        return self.get_historical_data(symbol, period, '1 W')
    
    def get_monthly_data(self, symbol: str, period: str = '5 Y') -> Optional[pd.DataFrame]:
        """
        Get monthly historical data
        
        Args:
            symbol: Stock symbol
            period: Time period (e.g., '5 Y', '10 Y')
        """
        return self.get_historical_data(symbol, period, '1 M')
    
    def get_multiple_symbols(self, symbols: List[str], duration: str = '1 D',
                            bar_size: str = '1 min') -> Dict[str, pd.DataFrame]:
        """
        Get historical data for multiple symbols
        
        Args:
            symbols: List of stock symbols
            duration: Data duration
            bar_size: Bar size
        """
        results = {}
        
        for symbol in symbols:
            self.logger.info(f"Fetching data for {symbol}")
            data = self.get_historical_data(symbol, duration, bar_size)
            
            if data is not None:
                results[symbol] = data
            
            # Small delay to avoid rate limiting
            time.sleep(0.1)
        
        self.logger.info(f"Retrieved data for {len(results)} out of {len(symbols)} symbols")
        return results
    
    def get_cached_data(self, symbol: str, duration: str = '1 D',
                       bar_size: str = '1 min', max_age_minutes: int = 5) -> Optional[pd.DataFrame]:
        """
        Get cached historical data if available and not expired
        
        Args:
            symbol: Stock symbol
            duration: Data duration
            bar_size: Bar size
            max_age_minutes: Maximum age of cached data in minutes
        """
        cache_key = f"{symbol}_{duration}_{bar_size}"
        
        if cache_key in self.data_cache:
            cached_item = self.data_cache[cache_key]
            age_minutes = (datetime.now() - cached_item['retrieved_at']).total_seconds() / 60
            
            if age_minutes <= max_age_minutes:
                self.logger.info(f"Using cached data for {symbol} (age: {age_minutes:.1f}min)")
                return cached_item['data']
        
        # Cache miss or expired, fetch fresh data
        return self.get_historical_data(symbol, duration, bar_size)
    
    def clear_cache(self):
        """Clear the data cache"""
        self.data_cache.clear()
        self.logger.info("Data cache cleared")
    
    def get_price_at_time(self, symbol: str, target_time: datetime,
                         duration: str = '1 D') -> Optional[float]:
        """
        Get price at a specific time using historical data
        
        Args:
            symbol: Stock symbol
            target_time: Target datetime
            duration: Data duration to search in
        """
        df = self.get_historical_data(symbol, duration, '1 min')
        
        if df is None or df.empty:
            return None
        
        # Find closest time
        df['time_diff'] = abs(df['timestamp'] - target_time)
        closest_row = df.loc[df['time_diff'].idxmin()]
        
        return float(closest_row['close'])

print("✓ Historical Data Handler class created")

✓ Historical Data Handler class created


### Market Data Handler

In [21]:
class MarketDataHandler:
    """Handle real-time market data from Interactive Brokers"""
    
    def __init__(self, ib_connection: IBConnection):
        """
        Initialise market data handler
        
        Args:
            ib_connection: IBConnection instance
        """
        self.ib_conn = ib_connection
        self.ib = ib_connection.ib
        self.logger = logging.getLogger('MarketDataHandler')
        self.subscribed_tickers = {}
        self.last_prices = {}
        
    def create_contract(self, symbol: str, exchange: str = 'SMART', 
                       currency: str = 'USD') -> Stock:
        """
        Create stock contract for IB
        
        Args:
            symbol: Stock symbol (e.g., 'MRK')
            exchange: Exchange (default: 'SMART')
            currency: Currency (default: 'USD')
        """
        return Stock(symbol, exchange, currency)
    
    def get_market_data(self, symbol: str, exchange: str = 'SMART',
                       currency: str = 'USD') -> Optional[Dict]:
        """
        Get real-time market data for a symbol
        
        Args:
            symbol: Stock symbol
            exchange: Exchange
            currency: Currency
        """
        if not self.ib_conn.is_connected():
            self.logger.error("Not connected to IB")
            return None
        
        try:
            contract = self.create_contract(symbol, exchange, currency)
            
            # Request market data
            ticker = self.ib.reqMktData(contract, '', False, False)
            self.ib.sleep(2)  # Allow time for data to arrive
            
            # Get ticker data
            if ticker and ticker.last and ticker.last > 0:
                market_data = {
                    'symbol': symbol,
                    'last': float(ticker.last),
                    'bid': float(ticker.bid) if ticker.bid and ticker.bid > 0 else None,
                    'ask': float(ticker.ask) if ticker.ask and ticker.ask > 0 else None,
                    'volume': int(ticker.volume) if ticker.volume else 0,
                    'high': float(ticker.high) if ticker.high else None,
                    'low': float(ticker.low) if ticker.low else None,
                    'close': float(ticker.close) if ticker.close else None,
                    'timestamp': datetime.now()
                }
                
                # Store last known price
                self.last_prices[symbol] = market_data['last']
                
                self.logger.info(f"Market data for {symbol}: £{market_data['last']:.2f}")
                return market_data
            else:
                self.logger.warning(f"No valid market data for {symbol}")
                return None
                
        except Exception as e:
            self.logger.error(f"Error getting market data for {symbol}: {e}")
            return None
        finally:
            # Cancel market data request to free up lines
            try:
                self.ib.cancelMktData(contract)
            except:
                pass
    
    def subscribe_to_ticker(self, symbol: str, exchange: str = 'SMART',
                           currency: str = 'USD') -> bool:
        """
        Subscribe to real-time ticker updates
        
        Args:
            symbol: Stock symbol
            exchange: Exchange
            currency: Currency
        """
        if not self.ib_conn.is_connected():
            return False
        
        try:
            contract = self.create_contract(symbol, exchange, currency)
            ticker = self.ib.reqMktData(contract, '', False, False)
            self.subscribed_tickers[symbol] = {
                'ticker': ticker,
                'contract': contract
            }
            self.logger.info(f"Subscribed to {symbol}")
            return True
        except Exception as e:
            self.logger.error(f"Error subscribing to {symbol}: {e}")
            return False
    
    def unsubscribe_from_ticker(self, symbol: str) -> bool:
        """
        Unsubscribe from ticker updates
        
        Args:
            symbol: Stock symbol
        """
        if symbol in self.subscribed_tickers:
            try:
                contract = self.subscribed_tickers[symbol]['contract']
                self.ib.cancelMktData(contract)
                del self.subscribed_tickers[symbol]
                self.logger.info(f"Unsubscribed from {symbol}")
                return True
            except Exception as e:
                self.logger.error(f"Error unsubscribing from {symbol}: {e}")
                return False
        return False
    
    def get_subscribed_data(self, symbol: str) -> Optional[Dict]:
        """
        Get current data for a subscribed ticker
        
        Args:
            symbol: Stock symbol
        """
        if symbol not in self.subscribed_tickers:
            self.logger.warning(f"Not subscribed to {symbol}")
            return None
        
        try:
            ticker = self.subscribed_tickers[symbol]['ticker']
            
            if ticker and ticker.last and ticker.last > 0:
                market_data = {
                    'symbol': symbol,
                    'last': float(ticker.last),
                    'bid': float(ticker.bid) if ticker.bid and ticker.bid > 0 else None,
                    'ask': float(ticker.ask) if ticker.ask and ticker.ask > 0 else None,
                    'volume': int(ticker.volume) if ticker.volume else 0,
                    'high': float(ticker.high) if ticker.high else None,
                    'low': float(ticker.low) if ticker.low else None,
                    'close': float(ticker.close) if ticker.close else None,
                    'timestamp': datetime.now()
                }
                
                # Store last known price
                self.last_prices[symbol] = market_data['last']
                
                return market_data
            else:
                return None
                
        except Exception as e:
            self.logger.error(f"Error getting subscribed data for {symbol}: {e}")
            return None
    
    def get_multiple_market_data(self, symbols: List[str]) -> Dict[str, Dict]:
        """
        Get market data for multiple symbols
        
        Args:
            symbols: List of stock symbols
        """
        results = {}
        
        for symbol in symbols:
            self.logger.info(f"Fetching market data for {symbol}")
            data = self.get_market_data(symbol)
            
            if data is not None:
                results[symbol] = data
            
            # Small delay to avoid overwhelming the API
            time.sleep(0.2)
        
        self.logger.info(f"Retrieved market data for {len(results)} out of {len(symbols)} symbols")
        return results
    
    def get_market_snapshot(self, symbol: str, exchange: str = 'SMART',
                           currency: str = 'USD') -> Optional[Dict]:
        """
        Get a quick market snapshot (single request)
        
        Args:
            symbol: Stock symbol
            exchange: Exchange
            currency: Currency
        """
        if not self.ib_conn.is_connected():
            self.logger.error("Not connected to IB")
            return None
        
        try:
            contract = self.create_contract(symbol, exchange, currency)
            
            # Request snapshot data
            ticker = self.ib.reqMktData(contract, '', True, False)  # Snapshot = True
            self.ib.sleep(1)  # Shorter wait for snapshot
            
            if ticker and ticker.last and ticker.last > 0:
                snapshot_data = {
                    'symbol': symbol,
                    'last': float(ticker.last),
                    'bid': float(ticker.bid) if ticker.bid and ticker.bid > 0 else None,
                    'ask': float(ticker.ask) if ticker.ask and ticker.ask > 0 else None,
                    'timestamp': datetime.now()
                }
                
                self.logger.info(f"Snapshot for {symbol}: £{snapshot_data['last']:.2f}")
                return snapshot_data
            else:
                self.logger.warning(f"No snapshot data for {symbol}")
                return None
                
        except Exception as e:
            self.logger.error(f"Error getting snapshot for {symbol}: {e}")
            return None
        finally:
            # Cancel snapshot request
            try:
                self.ib.cancelMktData(contract)
            except:
                pass
    
    def cleanup_subscriptions(self):
        """Cancel all active subscriptions"""
        for symbol in list(self.subscribed_tickers.keys()):
            self.unsubscribe_from_ticker(symbol)
        self.logger.info("All market data subscriptions cancelled")
    
    def get_last_known_price(self, symbol: str) -> Optional[float]:
        """
        Get last known price for a symbol
        
        Args:
            symbol: Stock symbol
        """
        return self.last_prices.get(symbol)
    
    def is_market_open(self) -> bool:
        """
        Check if market is currently open (basic US market hours)
        """
        now = datetime.now()
        # Basic US market hours check (9:30 AM - 4:00 PM ET)
        market_open = now.replace(hour=9, minute=30, second=0, microsecond=0)
        market_close = now.replace(hour=16, minute=0, second=0, microsecond=0)
        
        # Check if it's a weekday and within market hours
        is_weekday = now.weekday() < 5  # Monday = 0, Friday = 4
        is_market_hours = market_open <= now <= market_close
        
        return is_weekday and is_market_hours

print("✓ Market Data Handler class created")

✓ Market Data Handler class created


### Order Management

In [23]:
class OrderManager:
    """Handle order placement and management"""
    
    def __init__(self, ib_connection: IBConnection):
        """
        Initialise order manager
        
        Args:
            ib_connection: IBConnection instance
        """
        self.ib_conn = ib_connection
        self.ib = ib_connection.ib
        self.logger = logging.getLogger('OrderManager')
        self.orders_log = []
    
    def place_market_order(self, symbol: str, action: str, quantity: int,
                          exchange: str = 'SMART', currency: str = 'USD') -> bool:
        """
        Place a market order
        
        Args:
            symbol: Stock symbol
            action: 'BUY' or 'SELL'
            quantity: Number of shares
            exchange: Exchange
            currency: Currency
        """
        if not self.ib_conn.is_connected():
            self.logger.error("Not connected to IB")
            return False
        
        try:
            contract = Stock(symbol, exchange, currency)
            order = MarketOrder(action, quantity)
            
            # Place the order
            trade = self.ib.placeOrder(contract, order)
            
            if trade:
                order_info = {
                    'symbol': symbol,
                    'action': action,
                    'quantity': quantity,
                    'order_type': 'MARKET',
                    'status': trade.orderStatus.status,
                    'timestamp': datetime.now(),
                    'order_id': trade.order.orderId
                }
                
                self.orders_log.append(order_info)
                
                self.logger.info(f"✓ Market order placed: {action} {quantity} {symbol}")
                return True
            else:
                self.logger.error(f"Failed to place market order for {symbol}")
                return False
                
        except Exception as e:
            self.logger.error(f"Error placing market order for {symbol}: {e}")
            return False
    
    def place_limit_order(self, symbol: str, action: str, quantity: int,
                         limit_price: float, exchange: str = 'SMART',
                         currency: str = 'USD') -> bool:
        """
        Place a limit order
        
        Args:
            symbol: Stock symbol
            action: 'BUY' or 'SELL'
            quantity: Number of shares
            limit_price: Limit price
            exchange: Exchange
            currency: Currency
        """
        if not self.ib_conn.is_connected():
            self.logger.error("Not connected to IB")
            return False
        
        try:
            contract = Stock(symbol, exchange, currency)
            order = LimitOrder(action, quantity, limit_price)
            
            # Place the order
            trade = self.ib.placeOrder(contract, order)
            
            if trade:
                order_info = {
                    'symbol': symbol,
                    'action': action,
                    'quantity': quantity,
                    'order_type': 'LIMIT',
                    'limit_price': limit_price,
                    'status': trade.orderStatus.status,
                    'timestamp': datetime.now(),
                    'order_id': trade.order.orderId
                }
                
                self.orders_log.append(order_info)
                
                self.logger.info(f"✓ Limit order placed: {action} {quantity} {symbol} @ £{limit_price:.2f}")
                return True
            else:
                self.logger.error(f"Failed to place limit order for {symbol}")
                return False
                
        except Exception as e:
            self.logger.error(f"Error placing limit order for {symbol}: {e}")
            return False
    
    def get_open_orders(self) -> List[Dict]:
        """Get list of open orders"""
        if not self.ib_conn.is_connected():
            return []
        
        try:
            trades = self.ib.openTrades()
            open_orders = []
            
            for trade in trades:
                order_info = {
                    'symbol': trade.contract.symbol,
                    'action': trade.order.action,
                    'quantity': trade.order.totalQuantity,
                    'order_type': trade.order.orderType,
                    'status': trade.orderStatus.status,
                    'order_id': trade.order.orderId,
                    'filled': trade.orderStatus.filled,
                    'remaining': trade.orderStatus.remaining
                }
                
                if hasattr(trade.order, 'lmtPrice') and trade.order.lmtPrice:
                    order_info['limit_price'] = trade.order.lmtPrice
                
                open_orders.append(order_info)
            
            return open_orders
            
        except Exception as e:
            self.logger.error(f"Error getting open orders: {e}")
            return []
    
    def cancel_order(self, order_id: int) -> bool:
        """Cancel an order by ID"""
        if not self.ib_conn.is_connected():
            return False
        
        try:
            self.ib.cancelOrder(order_id)
            self.logger.info(f"Order {order_id} cancellation requested")
            return True
        except Exception as e:
            self.logger.error(f"Error cancelling order {order_id}: {e}")
            return False
    
    def get_orders_log(self) -> List[Dict]:
        """Get log of all placed orders"""
        return self.orders_log

print("✓ Order Manager class created")

✓ Order Manager class created


### Trading Strategy

In [24]:
class TradingStrategy:
    """Advanced trading strategy using both historical and real-time market data"""
    
    def __init__(self, name: str, short_window: int = 10, long_window: int = 30,
                 historical_data_handler: HistoricalDataHandler = None,
                 market_data_handler: MarketDataHandler = None):
        """
        Initialise trading strategy
        
        Args:
            name: Strategy name
            short_window: Short moving average period
            long_window: Long moving average period
            historical_data_handler: Historical data handler instance
            market_data_handler: Market data handler instance
        """
        self.name = name
        self.short_window = short_window
        self.long_window = long_window
        self.logger = logging.getLogger(f'Strategy-{name}')
        self.historical_data = historical_data_handler
        self.market_data = market_data_handler
        
        # Strategy state
        self.positions = {}
        self.technical_indicators = {}
        self.price_history = {}
        self.signal_history = {}
        
        # Performance tracking
        self.performance_metrics = {
            'total_trades': 0,
            'profitable_trades': 0,
            'total_pnl': 0.0,
            'win_rate': 0.0,
            'max_drawdown': 0.0,
            'sharpe_ratio': 0.0
        }
        
        # Risk management parameters
        self.max_position_size = 0.20  # Maximum 20% of capital per position
        self.stop_loss_pct = 0.05      # 5% stop loss
        self.take_profit_pct = 0.15    # 15% take profit
        
    def calculate_technical_indicators(self, symbol: str, df: pd.DataFrame) -> Dict:
        """
        Calculate comprehensive technical indicators from historical data
        
        Args:
            symbol: Stock symbol
            df: Historical price DataFrame
        """
        if df is None or df.empty:
            return {}
        
        indicators = {}
        
        try:
            # Moving averages
            if len(df) >= 5:
                indicators['sma_5'] = df['close'].rolling(window=5).mean().iloc[-1]
            if len(df) >= 10:
                indicators['sma_10'] = df['close'].rolling(window=10).mean().iloc[-1]
            if len(df) >= 20:
                indicators['sma_20'] = df['close'].rolling(window=20).mean().iloc[-1]
            if len(df) >= 50:
                indicators['sma_50'] = df['close'].rolling(window=50).mean().iloc[-1]
            
            # Exponential moving averages
            if len(df) >= 12:
                indicators['ema_12'] = df['close'].ewm(span=12).mean().iloc[-1]
            if len(df) >= 26:
                indicators['ema_26'] = df['close'].ewm(span=26).mean().iloc[-1]
                
            # MACD calculation
            if 'ema_12' in indicators and 'ema_26' in indicators:
                macd_line = indicators['ema_12'] - indicators['ema_26']
                indicators['macd'] = macd_line
                
                # MACD signal line (9-period EMA of MACD)
                if len(df) >= 26:
                    macd_series = df['close'].ewm(span=12).mean() - df['close'].ewm(span=26).mean()
                    indicators['macd_signal'] = macd_series.ewm(span=9).mean().iloc[-1]
                    indicators['macd_histogram'] = macd_line - indicators['macd_signal']
            
            # RSI calculation
            if len(df) >= 14:
                delta = df['close'].diff()
                gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
                loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
                rs = gain / loss
                indicators['rsi'] = 100 - (100 / (1 + rs.iloc[-1]))
            
            # Bollinger Bands
            if len(df) >= 20:
                sma_20 = df['close'].rolling(window=20).mean()
                std_20 = df['close'].rolling(window=20).std()
                indicators['bb_upper'] = (sma_20 + (std_20 * 2)).iloc[-1]
                indicators['bb_lower'] = (sma_20 - (std_20 * 2)).iloc[-1]
                indicators['bb_middle'] = sma_20.iloc[-1]
                indicators['bb_width'] = ((indicators['bb_upper'] - indicators['bb_lower']) / indicators['bb_middle']) * 100
            
            # Volume indicators
            if len(df) >= 10:
                indicators['volume_sma'] = df['volume'].rolling(window=10).mean().iloc[-1]
                indicators['volume_ratio'] = df['volume'].iloc[-1] / indicators['volume_sma']
            
            # Volatility measures
            if len(df) >= 20:
                returns = df['close'].pct_change()
                indicators['volatility'] = returns.rolling(window=20).std().iloc[-1] * 100
                indicators['atr'] = self.calculate_atr(df, 14)
            
            # Price momentum indicators
            if len(df) >= 5:
                indicators['momentum_5'] = ((df['close'].iloc[-1] - df['close'].iloc[-5]) / df['close'].iloc[-5]) * 100
            if len(df) >= 10:
                indicators['momentum_10'] = ((df['close'].iloc[-1] - df['close'].iloc[-10]) / df['close'].iloc[-10]) * 100
            
            # Support and resistance levels
            if len(df) >= 20:
                recent_highs = df['high'].rolling(window=20).max()
                recent_lows = df['low'].rolling(window=20).min()
                indicators['resistance'] = recent_highs.iloc[-1]
                indicators['support'] = recent_lows.iloc[-1]
            
            # Market strength indicators
            if len(df) >= 14:
                # Williams %R
                highest_high = df['high'].rolling(window=14).max().iloc[-1]
                lowest_low = df['low'].rolling(window=14).min().iloc[-1]
                current_close = df['close'].iloc[-1]
                indicators['williams_r'] = ((highest_high - current_close) / (highest_high - lowest_low)) * -100
            
            self.technical_indicators[symbol] = indicators
            
        except Exception as e:
            self.logger.error(f"Error calculating indicators for {symbol}: {e}")
        
        return indicators
    
    def calculate_atr(self, df: pd.DataFrame, period: int = 14) -> float:
        """
        Calculate Average True Range
        
        Args:
            df: Historical price DataFrame
            period: ATR period
        """
        try:
            high_low = df['high'] - df['low']
            high_close = abs(df['high'] - df['close'].shift())
            low_close = abs(df['low'] - df['close'].shift())
            
            true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
            atr = true_range.rolling(window=period).mean().iloc[-1]
            
            return atr
        except:
            return 0.0
    
    def get_enhanced_market_analysis(self, symbol: str) -> Optional[Dict]:
        """
        Get comprehensive market analysis combining historical and real-time data
        
        Args:
            symbol: Stock symbol
        """
        analysis = {}
        
        try:
            # Get historical data for technical analysis
            if self.historical_data:
                df = self.historical_data.get_intraday_data(symbol, days=3)
                if df is not None and not df.empty:
                    analysis['historical_data'] = df
                    analysis['technical_indicators'] = self.calculate_technical_indicators(symbol, df)
                    
                    # Get daily data for longer-term trends
                    daily_df = self.historical_data.get_daily_data(symbol, period='1 M')
                    if daily_df is not None and not daily_df.empty:
                        analysis['daily_trend'] = self.calculate_trend_strength(daily_df)
            
            # Get real-time market data for current conditions
            if self.market_data:
                real_time_data = self.market_data.get_market_data(symbol)
                if real_time_data:
                    analysis.update(real_time_data)
                    
                    # Check market conditions
                    analysis['market_open'] = self.market_data.is_market_open()
            
            # Fallback to historical data if real-time unavailable
            if 'last' not in analysis and 'historical_data' in analysis:
                df = analysis['historical_data']
                if not df.empty:
                    analysis['last'] = float(df.iloc[-1]['close'])
                    analysis['volume'] = int(df.iloc[-1]['volume'])
                    analysis['symbol'] = symbol
                    analysis['timestamp'] = datetime.now()
            
            return analysis if analysis else None
            
        except Exception as e:
            self.logger.error(f"Error getting enhanced analysis for {symbol}: {e}")
            return None
    
    def calculate_trend_strength(self, df: pd.DataFrame) -> Dict:
        """
        Calculate trend strength from daily data
        
        Args:
            df: Daily price DataFrame
        """
        trend_data = {}
        
        try:
            if len(df) >= 20:
                sma_20 = df['close'].rolling(window=20).mean()
                current_price = df['close'].iloc[-1]
                sma_current = sma_20.iloc[-1]
                
                # Trend direction
                trend_data['direction'] = 'bullish' if current_price > sma_current else 'bearish'
                
                # Trend strength (how far from SMA)
                trend_data['strength'] = abs((current_price - sma_current) / sma_current) * 100
                
                # Price momentum over 5, 10, 20 days
                if len(df) >= 5:
                    trend_data['momentum_5d'] = ((df['close'].iloc[-1] - df['close'].iloc[-5]) / df['close'].iloc[-5]) * 100
                if len(df) >= 10:
                    trend_data['momentum_10d'] = ((df['close'].iloc[-1] - df['close'].iloc[-10]) / df['close'].iloc[-10]) * 100
                if len(df) >= 20:
                    trend_data['momentum_20d'] = ((df['close'].iloc[-1] - df['close'].iloc[-20]) / df['close'].iloc[-20]) * 100
                    
        except Exception as e:
            self.logger.error(f"Error calculating trend strength: {e}")
        
        return trend_data
    
    def update_price_history(self, symbol: str, price: float):
        """Update price history for real-time calculations"""
        if symbol not in self.price_history:
            self.price_history[symbol] = []
        
        self.price_history[symbol].append(price)
        
        # Keep only required history
        max_length = max(self.short_window, self.long_window) + 20
        if len(self.price_history[symbol]) > max_length:
            self.price_history[symbol] = self.price_history[symbol][-max_length:]
    
    def calculate_sma(self, symbol: str, window: int) -> Optional[float]:
        """Calculate simple moving average from price history"""
        if symbol not in self.price_history:
            return None
        
        prices = self.price_history[symbol]
        if len(prices) < window:
            return None
        
        return sum(prices[-window:]) / window
    
    def evaluate_buy_signals(self, symbol: str, market_data: Dict, indicators: Dict) -> List[bool]:
        """
        Evaluate multiple buy signals
        
        Args:
            symbol: Stock symbol
            market_data: Market data dictionary
            indicators: Technical indicators
        """
        signals = []
        price = market_data['last']
        
        # Signal 1: SMA crossover (traditional)
        if 'sma_10' in indicators and 'sma_20' in indicators:
            signals.append(indicators['sma_10'] > indicators['sma_20'])
        
        # Signal 2: RSI oversold
        if 'rsi' in indicators:
            signals.append(20 < indicators['rsi'] < 35)  # Slightly oversold but not extreme
        
        # Signal 3: Price above Bollinger Band lower
        if 'bb_lower' in indicators and 'bb_middle' in indicators:
            signals.append(price > indicators['bb_lower'] and price < indicators['bb_middle'])
        
        # Signal 4: MACD bullish
        if 'macd' in indicators and 'macd_signal' in indicators:
            signals.append(indicators['macd'] > indicators['macd_signal'])
        
        # Signal 5: Volume confirmation
        if 'volume_ratio' in indicators:
            signals.append(indicators['volume_ratio'] > 1.3)  # 30% above average
        
        # Signal 6: Positive momentum
        if 'momentum_5' in indicators:
            signals.append(indicators['momentum_5'] > 0.5)
        
        # Signal 7: Williams %R oversold
        if 'williams_r' in indicators:
            signals.append(-80 < indicators['williams_r'] < -50)
        
        return signals
    
    def evaluate_sell_signals(self, symbol: str, market_data: Dict, indicators: Dict) -> List[bool]:
        """
        Evaluate multiple sell signals
        
        Args:
            symbol: Stock symbol
            market_data: Market data dictionary
            indicators: Technical indicators
        """
        signals = []
        price = market_data['last']
        
        # Signal 1: SMA crossover (traditional)
        if 'sma_10' in indicators and 'sma_20' in indicators:
            signals.append(indicators['sma_10'] < indicators['sma_20'])
        
        # Signal 2: RSI overbought
        if 'rsi' in indicators:
            signals.append(indicators['rsi'] > 70)
        
        # Signal 3: Price near Bollinger Band upper
        if 'bb_upper' in indicators:
            signals.append(price > indicators['bb_upper'] * 0.98)
        
        # Signal 4: MACD bearish
        if 'macd' in indicators and 'macd_signal' in indicators:
            signals.append(indicators['macd'] < indicators['macd_signal'])
        
        # Signal 5: Negative momentum
        if 'momentum_5' in indicators:
            signals.append(indicators['momentum_5'] < -1.0)
        
        # Signal 6: Williams %R overbought
        if 'williams_r' in indicators:
            signals.append(indicators['williams_r'] > -20)
        
        # Signal 7: High volatility (risk management)
        if 'volatility' in indicators:
            signals.append(indicators['volatility'] > 5.0)  # High volatility warning
        
        return signals
    
    def check_risk_management(self, symbol: str, market_data: Dict) -> Dict:
        """
        Check risk management conditions
        
        Args:
            symbol: Stock symbol
            market_data: Market data dictionary
        """
        risk_check = {'stop_loss': False, 'take_profit': False, 'risk_level': 'normal'}
        
        position = self.get_position(symbol)
        if position['shares'] > 0:
            current_price = market_data['last']
            entry_price = position['avg_price']
            
            # Calculate P&L percentage
            pnl_pct = ((current_price - entry_price) / entry_price) * 100
            
            # Stop loss check
            if pnl_pct <= -self.stop_loss_pct * 100:
                risk_check['stop_loss'] = True
                risk_check['risk_level'] = 'high'
            
            # Take profit check
            if pnl_pct >= self.take_profit_pct * 100:
                risk_check['take_profit'] = True
            
            # Track current P&L
            risk_check['current_pnl_pct'] = pnl_pct
            
        return risk_check
    
    def should_buy(self, market_data: Dict) -> bool:
        """Enhanced buy logic using comprehensive analysis"""
        symbol = market_data['symbol']
        price = market_data['last']
        
        # Check if we already have a position
        position = self.get_position(symbol)
        if position['shares'] > 0:
            return False
        
        self.update_price_history(symbol, price)
        
        # Get enhanced analysis
        enhanced_analysis = self.get_enhanced_market_analysis(symbol)
        
        if enhanced_analysis and 'technical_indicators' in enhanced_analysis:
            indicators = enhanced_analysis['technical_indicators']
            
            # Evaluate buy signals
            buy_signals = self.evaluate_buy_signals(symbol, market_data, indicators)
            signal_count = sum(buy_signals)
            
            # Check daily trend if available
            trend_confirmation = True
            if 'daily_trend' in enhanced_analysis:
                trend = enhanced_analysis['daily_trend']
                if 'direction' in trend:
                    trend_confirmation = trend['direction'] == 'bullish'
            
            # Store signal history for analysis
            if symbol not in self.signal_history:
                self.signal_history[symbol] = []
            
            self.signal_history[symbol].append({
                'timestamp': datetime.now(),
                'signal_count': signal_count,
                'signals': buy_signals,
                'action': 'buy_evaluation',
                'price': price
            })
            
            # Buy decision: need majority of signals + trend confirmation
            required_signals = max(2, len(buy_signals) // 2)
            decision = signal_count >= required_signals and trend_confirmation
            
            if decision:
                self.logger.info(f"BUY signal for {symbol}: {signal_count}/{len(buy_signals)} signals")
            
            return decision
        
        else:
            # Fallback to simple SMA logic
            short_sma = self.calculate_sma(symbol, self.short_window)
            long_sma = self.calculate_sma(symbol, self.long_window)
            
            if short_sma and long_sma:
                return short_sma > long_sma
        
        return False
    
    def should_sell(self, symbol: str, market_data: Dict) -> bool:
        """Enhanced sell logic using comprehensive analysis and risk management"""
        price = market_data['last']
        
        # Check if we have a position to sell
        position = self.get_position(symbol)
        if position['shares'] <= 0:
            return False
        
        self.update_price_history(symbol, price)
        
        # Risk management check first
        risk_check = self.check_risk_management(symbol, market_data)
        if risk_check['stop_loss'] or risk_check['take_profit']:
            action = 'STOP_LOSS' if risk_check['stop_loss'] else 'TAKE_PROFIT'
            self.logger.info(f"{action} triggered for {symbol} at {risk_check['current_pnl_pct']:+.2f}%")
            return True
        
        # Get enhanced analysis
        enhanced_analysis = self.get_enhanced_market_analysis(symbol)
        
        if enhanced_analysis and 'technical_indicators' in enhanced_analysis:
            indicators = enhanced_analysis['technical_indicators']
            
            # Evaluate sell signals
            sell_signals = self.evaluate_sell_signals(symbol, market_data, indicators)
            signal_count = sum(sell_signals)
            
            # Store signal history
            if symbol not in self.signal_history:
                self.signal_history[symbol] = []
            
            self.signal_history[symbol].append({
                'timestamp': datetime.now(),
                'signal_count': signal_count,
                'signals': sell_signals,
                'action': 'sell_evaluation',
                'price': price
            })
            
            # Sell decision: need majority of signals
            required_signals = max(2, len(sell_signals) // 2)
            decision = signal_count >= required_signals
            
            if decision:
                self.logger.info(f"SELL signal for {symbol}: {signal_count}/{len(sell_signals)} signals")
            
            return decision
        
        else:
            # Fallback to simple SMA logic
            short_sma = self.calculate_sma(symbol, self.short_window)
            long_sma = self.calculate_sma(symbol, self.long_window)
            
            if short_sma and long_sma:
                return short_sma < long_sma
        
        return False
    
    def calculate_position_size(self, symbol: str, price: float, 
                              available_capital: float) -> int:
        """
        Calculate position size with risk management
        
        Args:
            symbol: Stock symbol
            price: Current price
            available_capital: Available capital
        """
        # Base position size (10% of capital)
        base_target_value = available_capital * 0.10
        
        # Adjust based on volatility if available
        if symbol in self.technical_indicators:
            indicators = self.technical_indicators[symbol]
            
            # Reduce position size for high volatility stocks
            if 'volatility' in indicators:
                volatility = indicators['volatility']
                if volatility > 3.0:  # High volatility
                    base_target_value *= 0.7  # Reduce by 30%
                elif volatility < 1.0:  # Low volatility
                    base_target_value *= 1.2  # Increase by 20%
            
            # Adjust based on trend strength
            if 'daily_trend' in indicators and 'strength' in indicators['daily_trend']:
                strength = indicators['daily_trend']['strength']
                if strength > 5.0:  # Strong trend
                    base_target_value *= 1.1  # Slight increase
        
        # Apply maximum position size limit
        max_target_value = available_capital * self.max_position_size
        target_value = min(base_target_value, max_target_value)
        
        shares = int(target_value / price)
        return max(1, shares)  # At least 1 share
    
    def update_position(self, symbol: str, action: str, quantity: int, price: float):
        """Update position tracking with P&L calculation"""
        if symbol not in self.positions:
            self.positions[symbol] = {'shares': 0, 'avg_price': 0.0, 'total_pnl': 0.0}
        
        position = self.positions[symbol]
        
        if action == 'BUY':
            total_cost = (position['shares'] * position['avg_price']) + (quantity * price)
            position['shares'] += quantity
            position['avg_price'] = total_cost / position['shares'] if position['shares'] > 0 else 0
            
        elif action == 'SELL':
            # Calculate P&L for this trade
            if position['shares'] > 0:
                trade_pnl = (price - position['avg_price']) * quantity
                position['total_pnl'] += trade_pnl
                
                # Update performance metrics
                self.performance_metrics['total_trades'] += 1
                if trade_pnl > 0:
                    self.performance_metrics['profitable_trades'] += 1
                
                self.performance_metrics['total_pnl'] += trade_pnl
                self.performance_metrics['win_rate'] = (
                    self.performance_metrics['profitable_trades'] / 
                    self.performance_metrics['total_trades']
                ) * 100
            
            position['shares'] -= quantity
            if position['shares'] <= 0:
                position['shares'] = 0
                position['avg_price'] = 0.0
    
    def get_position(self, symbol: str) -> Dict:
        """Get current position for symbol"""
        return self.positions.get(symbol, {'shares': 0, 'avg_price': 0.0, 'total_pnl': 0.0})
    
    def get_performance_summary(self) -> Dict:
        """Get comprehensive strategy performance summary"""
        summary = self.performance_metrics.copy()
        
        # Add current positions value
        total_position_value = 0.0
        for symbol, position in self.positions.items():
            if position['shares'] > 0:
                # Use last known price or get current price
                current_price = 0.0
                if self.market_data:
                    current_data = self.market_data.get_last_known_price(symbol)
                    if current_data:
                        current_price = current_data
                
                if current_price > 0:
                    position_value = position['shares'] * current_price
                    unrealized_pnl = (current_price - position['avg_price']) * position['shares']
                    total_position_value += position_value
                    
                    summary[f'{symbol}_position_value'] = position_value
                    summary[f'{symbol}_unrealized_pnl'] = unrealized_pnl
        
        summary['total_position_value'] = total_position_value
        
        return summary
    
    def set_data_handlers(self, historical_data_handler: HistoricalDataHandler = None,
                         market_data_handler: MarketDataHandler = None):
        """
        Set data handlers for enhanced analysis
        
        Args:
            historical_data_handler: Historical data handler instance
            market_data_handler: Market data handler instance
        """
        self.historical_data = historical_data_handler
        self.market_data = market_data_handler
        self.logger.info("Data handlers configured for enhanced analysis")
    
    def get_strategy_status(self, symbol: str) -> Dict:
        """
        Get comprehensive strategy status for a symbol
        
        Args:
            symbol: Stock symbol
        """
        status = {
            'symbol': symbol,
            'position': self.get_position(symbol),
            'technical_indicators': self.technical_indicators.get(symbol, {}),
            'price_history_length': len(self.price_history.get(symbol, [])),
            'recent_signals': self.signal_history.get(symbol, [])[-5:] if symbol in self.signal_history else []
        }
        
        return status
    
    def get_market_overview(self, symbols: List[str]) -> Dict:
        """
        Get market overview for all tracked symbols
        
        Args:
            symbols: List of symbols to analyse
        """
        overview = {
            'total_positions': 0,
            'total_value': 0.0,
            'symbols_analysis': {}
        }
        
        for symbol in symbols:
            try:
                status = self.get_strategy_status(symbol)
                overview['symbols_analysis'][symbol] = status
                
                if status['position']['shares'] > 0:
                    overview['total_positions'] += 1
                    
            except Exception as e:
                self.logger.error(f"Error getting overview for {symbol}: {e}")
        
        return overview

print("✓ Trading Strategy with Historical and Market Data created")

✓ Trading Strategy with Historical and Market Data created


### Trading Bot

In [25]:
class IBTradingBot:
    """Main Interactive Brokers trading bot with strategy integration"""
    
    def __init__(self, environment: str = 'paper', strategy: TradingStrategy = None):
        """
        Initialise trading bot
        
        Args:
            environment: 'paper' or 'live'
            strategy: Trading strategy instance
        """
        self.environment = environment
        self.logger = logging.getLogger(f'TradingBot-{environment}')
        
        # Initialize components
        self.ib_conn = IBConnection(environment)
        self.historical_data = None
        self.market_data = None
        self.order_manager = None
        self.strategy = strategy or TradingStrategy("Default_SMA", 10, 30)
        
        # Bot configuration
        self.symbols = ['ABCL','ALLO','ANNX','ANRO','ANVS','APLM','APLT','APVO','ARAY','ARBE',
    'ATEN','ATAI','ATHA','AUTL','AVIR','CABA','CAMP','CARM','CATX','CCCC',
    'CELU','CERS','CGTX','CHRS','CLDI','CLLS','CLRB','CLSD','CLYM','CMPS',
    'CMPX','CMTL','COCP','CRBU','CRDF','CRDL','CRIS','CTMX','CTOR','CTXR',
    'CUE','CURR','DARE','DTIL','EDIT','ELDN','EQ','ERAS','ERNA','ESLA',
    'ESPR','EVGN','FATE','FBLG']  # Default symbols to trade
        self.is_running = False
        self.update_interval = 10  # seconds between updates
        
        # Performance tracking
        self.start_time = None
        self.trades_executed = 0
        
        self.logger.info(f"Trading bot initialised for {environment} trading")
        self.logger.info(f"Strategy: {self.strategy.name}")
    
    def add_symbol(self, symbol: str):
        """Add symbol to trading list"""
        if symbol not in self.symbols:
            self.symbols.append(symbol)
            self.logger.info(f"Added {symbol} to trading symbols")
    
    def remove_symbol(self, symbol: str):
        """Remove symbol from trading list"""
        if symbol in self.symbols:
            self.symbols.remove(symbol)
            self.logger.info(f"Removed {symbol} from trading symbols")
    
    def set_symbols(self, symbols: List[str]):
        """Set list of symbols to trade"""
        self.symbols = symbols
        self.logger.info(f"Updated trading symbols: {symbols}")
    
    def connect(self) -> bool:
        """Connect to Interactive Brokers and initialise all handlers"""
        success = self.ib_conn.connect_sync(timeout=15)
        
        if success:
            # Initialize data handlers
            self.historical_data = HistoricalDataHandler(self.ib_conn)
            self.market_data = MarketDataHandler(self.ib_conn)
            self.order_manager = OrderManager(self.ib_conn)
            
            # Configure strategy with data handlers for enhanced analysis
            self.strategy.set_data_handlers(self.historical_data, self.market_data)
            
            self.logger.info("Trading bot connected successfully")
            
            # Display account information
            account_info = self.ib_conn.get_account_summary()
            if account_info:
                buying_power = account_info.get('BuyingPower', 'N/A')
                net_liquidation = account_info.get('NetLiquidation', 'N/A')
                self.logger.info(f"Account Net Liquidation: £{net_liquidation}")
                self.logger.info(f"Buying Power: £{buying_power}")
            
            return True
        else:
            self.logger.error("Failed to connect to IB")
            return False
    
    def disconnect(self):
        """Disconnect from Interactive Brokers and cleanup"""
        self.is_running = False
        
        # Cleanup market data subscriptions
        if self.market_data:
            self.market_data.cleanup_subscriptions()
        
        # Disconnect from IB
        if self.ib_conn:
            self.ib_conn.disconnect()
            
        self.logger.info("Trading bot disconnected")
    
    def get_enhanced_market_analysis(self, symbol: str) -> Optional[Dict]:
        """
        Get enhanced market analysis using both historical and real-time data
        
        Args:
            symbol: Stock symbol to analyse
        """
        try:
            # Use strategy's enhanced analysis method
            analysis = self.strategy.get_enhanced_market_analysis(symbol)
            
            if analysis:
                self.logger.info(
                    f"Enhanced analysis for {symbol}: £{analysis['last']:.2f} "
                    f"({analysis.get('price_change_pct', 0):+.2f}%)"
                )
                return analysis
            else:
                # Fallback to basic historical analysis
                return self.get_market_analysis(symbol)
                
        except Exception as e:
            self.logger.error(f"Error getting enhanced analysis for {symbol}: {e}")
            return self.get_market_analysis(symbol)
    
    def get_market_analysis(self, symbol: str) -> Optional[Dict]:
        """
        Get basic market analysis data for trading decisions (fallback method)
        
        Args:
            symbol: Stock symbol to analyse
        """
        try:
            # Get recent historical data for analysis
            df = self.historical_data.get_intraday_data(symbol, days=2)
            
            if df is None or df.empty:
                self.logger.warning(f"No historical data available for {symbol}")
                return None
            
            # Calculate current metrics
            latest_price = float(df.iloc[-1]['close'])
            volume = int(df.iloc[-1]['volume'])
            
            # Calculate moving averages if enough data
            if len(df) >= 20:
                ma_20 = df['close'].rolling(window=20).mean().iloc[-1]
            else:
                ma_20 = latest_price
            
            if len(df) >= 50:
                ma_50 = df['close'].rolling(window=50).mean().iloc[-1]
            else:
                ma_50 = latest_price
            
            # Price change from previous close
            if len(df) >= 2:
                price_change = latest_price - float(df.iloc[-2]['close'])
                price_change_pct = (price_change / float(df.iloc[-2]['close'])) * 100
            else:
                price_change = 0
                price_change_pct = 0
            
            analysis_data = {
                'symbol': symbol,
                'last': latest_price,
                'volume': volume,
                'price_change': price_change,
                'price_change_pct': price_change_pct,
                'ma_20': float(ma_20),
                'ma_50': float(ma_50),
                'timestamp': datetime.now(),
                'historical_data': df
            }
            
            self.logger.info(f"Analysis for {symbol}: £{latest_price:.2f} ({price_change_pct:+.2f}%)")
            return analysis_data
            
        except Exception as e:
            self.logger.error(f"Error getting market analysis for {symbol}: {e}")
            return None
    
    def execute_trade_logic(self, symbol: str, market_data: Dict) -> bool:
        """Execute enhanced trading logic using the strategy's advanced features"""
        try:
            # Get account information for position sizing
            if self.environment == 'simulator':
                # Use simulated balance
                buying_power = getattr(self, 'simulated_balance', 10000.0)
            else:
                # Get real account information
                account_info = self.ib_conn.get_account_summary()
                buying_power = float(account_info.get('BuyingPower', '0')) if account_info else 0
            
            # Check for buy signal using enhanced strategy
            if self.strategy.should_buy(market_data):
                if buying_power > 0:
                    quantity = self.strategy.calculate_position_size(
                        symbol, market_data['last'], buying_power
                    )
                    
                    if quantity > 0:
                        success = self.order_manager.place_market_order(
                            symbol, 'BUY', quantity
                        )
                        
                        if success:
                            self.strategy.update_position(
                                symbol, 'BUY', quantity, market_data['last']
                            )
                            
                            # Update simulated balance if in simulator
                            if self.environment == 'simulator':
                                trade_cost = quantity * market_data['last']
                                self.simulated_balance -= trade_cost
                            
                            self.trades_executed += 1
                            self.logger.info(f"BUY executed: {quantity} shares of {symbol}")
                            return True
            
            # Check for sell signal using enhanced strategy
            elif self.strategy.should_sell(symbol, market_data):
                position = self.strategy.get_position(symbol)
                
                if position['shares'] > 0:
                    success = self.order_manager.place_market_order(
                        symbol, 'SELL', position['shares']
                    )
                    
                    if success:
                        self.strategy.update_position(
                            symbol, 'SELL', position['shares'], market_data['last']
                        )
                        
                        # Update simulated balance if in simulator
                        if self.environment == 'simulator':
                            trade_value = position['shares'] * market_data['last']
                            self.simulated_balance += trade_value
                        
                        self.trades_executed += 1
                        self.logger.info(f"SELL executed: {position['shares']} shares of {symbol}")
                        return True
            
            return False
            
        except Exception as e:
            self.logger.error(f"Error in trade logic for {symbol}: {e}")
            return False
    
    def run_single_cycle(self):
        """Run a single enhanced trading cycle"""
        # Check connection (handle both real and simulated bots)
        if self.ib_conn and not self.ib_conn.is_connected():
            self.logger.error("Not connected to IB")
            return False
        elif not self.ib_conn and self.environment != 'simulator':
            self.logger.error("No IB connection available")
            return False
        
        trades_this_cycle = 0
        
        for symbol in self.symbols:
            try:
                # Get enhanced market analysis
                market_data = self.get_enhanced_market_analysis(symbol)
                
                if market_data:
                    # Execute trading logic using enhanced strategy
                    if self.execute_trade_logic(symbol, market_data):
                        trades_this_cycle += 1
                        
                        # Log strategy status after trade
                        status = self.strategy.get_strategy_status(symbol)
                        if status['technical_indicators']:
                            indicators = status['technical_indicators']
                            self.logger.info(
                                f"{symbol} indicators - RSI: {indicators.get('rsi', 'N/A'):.1f}, "
                                f"MACD: {indicators.get('macd', 'N/A'):.3f}, "
                                f"Volatility: {indicators.get('volatility', 'N/A'):.2f}%"
                            )
                else:
                    self.logger.warning(f"No market data available for {symbol}")
                    
            except Exception as e:
                self.logger.error(f"Error processing {symbol}: {e}")
        
        # Log cycle summary with performance metrics
        if trades_this_cycle > 0:
            self.logger.info(f"Cycle completed: {trades_this_cycle} trades executed")
            
            # Show performance summary after trades
            perf = self.strategy.get_performance_summary()
            if perf['total_trades'] > 0:
                self.logger.info(
                    f"Performance - Total Trades: {perf['total_trades']}, "
                    f"Win Rate: {perf['win_rate']:.1f}%, "
                    f"Total P&L: £{perf['total_pnl']:.2f}"
                )
        
        return True
    
    def start_trading(self, max_cycles: int = None):
        """
        Start automated trading with enhanced strategy monitoring
        
        Args:
            max_cycles: Maximum number of cycles to run (None for continuous)
        """
        # Check connection (handle both real and simulated bots)
        if self.ib_conn and not self.ib_conn.is_connected():
            self.logger.error("Must connect to IB before starting trading")
            return False
        elif not self.ib_conn and self.environment != 'simulator':
            self.logger.error("No IB connection available")
            return False
        
        self.is_running = True
        self.start_time = datetime.now()
        cycle_count = 0
        
        self.logger.info("Starting automated trading...")
        self.logger.info(f"Symbols: {self.symbols}")
        self.logger.info(f"Strategy: {self.strategy.name}")
        self.logger.info(f"Update interval: {self.update_interval} seconds")
        
        # Display strategy configuration
        self.logger.info(
            f"Strategy config - Short: {self.strategy.short_window}, "
            f"Long: {self.strategy.long_window}, "
            f"Stop Loss: {self.strategy.stop_loss_pct*100:.1f}%, "
            f"Take Profit: {self.strategy.take_profit_pct*100:.1f}%"
        )
        
        try:
            while self.is_running:
                cycle_count += 1
                
                if max_cycles and cycle_count > max_cycles:
                    self.logger.info(f"Reached maximum cycles ({max_cycles})")
                    break
                
                self.logger.info(f"--- Cycle {cycle_count} ---")
                
                # Run enhanced trading cycle
                self.run_single_cycle()
                
                # Show market overview every 5 cycles
                if cycle_count % 5 == 0:
                    try:
                        overview = self.strategy.get_market_overview(self.symbols)
                        self.logger.info(
                            f"Market Overview - Active Positions: {overview['total_positions']}, "
                            f"Symbols Tracked: {len(overview['symbols_analysis'])}"
                        )
                    except Exception as e:
                        self.logger.error(f"Error getting market overview: {e}")
                
                # Sleep before next cycle
                if self.is_running:
                    time.sleep(self.update_interval)
                
        except KeyboardInterrupt:
            self.logger.info("Trading stopped by user")
        except Exception as e:
            self.logger.error(f"Error in trading loop: {e}")
        finally:
            self.is_running = False
            self.print_enhanced_summary()
    
    def stop_trading(self):
        """Stop automated trading"""
        self.is_running = False
        self.logger.info("Trading stop requested")
    
    def print_enhanced_summary(self):
        """Print enhanced trading session summary with detailed performance metrics"""
        if self.start_time:
            duration = datetime.now() - self.start_time
            self.logger.info("="*50)
            self.logger.info("ENHANCED TRADING SESSION SUMMARY")
            self.logger.info("="*50)
            self.logger.info(f"Environment: {self.environment.upper()}")
            self.logger.info(f"Duration: {duration}")
            self.logger.info(f"Total trades executed: {self.trades_executed}")
            
            # Enhanced strategy performance
            perf = self.strategy.get_performance_summary()
            self.logger.info(f"Strategy: {self.strategy.name}")
            
            if perf['total_trades'] > 0:
                self.logger.info(f"Win Rate: {perf['win_rate']:.1f}%")
                self.logger.info(f"Total P&L: £{perf['total_pnl']:.2f}")
                self.logger.info(f"Profitable Trades: {perf['profitable_trades']}/{perf['total_trades']}")
            
            # Enhanced positions summary
            self.logger.info("Current Positions:")
            total_position_value = 0.0
            
            for symbol, position in self.strategy.positions.items():
                if position['shares'] > 0:
                    position_value = position['shares'] * position['avg_price']
                    total_position_value += position_value
                    
                    # Get current price for unrealised P&L
                    try:
                        current_data = self.get_enhanced_market_analysis(symbol)
                        if current_data:
                            current_price = current_data['last']
                            unrealised_pnl = (current_price - position['avg_price']) * position['shares']
                            unrealised_pct = (unrealised_pnl / position_value) * 100 if position_value > 0 else 0
                            
                            self.logger.info(
                                f"  {symbol}: {position['shares']} shares @ £{position['avg_price']:.2f} "
                                f"(Current: £{current_price:.2f}, P&L: £{unrealised_pnl:+.2f} {unrealised_pct:+.1f}%)"
                            )
                        else:
                            self.logger.info(f"  {symbol}: {position['shares']} shares @ £{position['avg_price']:.2f}")
                    except:
                        self.logger.info(f"  {symbol}: {position['shares']} shares @ £{position['avg_price']:.2f}")
                else:
                    self.logger.info(f"  {symbol}: No position")
            
            if total_position_value > 0:
                self.logger.info(f"Total Position Value: £{total_position_value:.2f}")
            
            # Technical indicators summary
            self.logger.info("Technical Indicators Summary:")
            for symbol in self.symbols:
                try:
                    status = self.strategy.get_strategy_status(symbol)
                    indicators = status['technical_indicators']
                    
                    if indicators:
                        summary_indicators = []
                        if 'rsi' in indicators:
                            summary_indicators.append(f"RSI: {indicators['rsi']:.1f}")
                        if 'macd' in indicators:
                            summary_indicators.append(f"MACD: {indicators['macd']:.3f}")
                        if 'volatility' in indicators:
                            summary_indicators.append(f"Vol: {indicators['volatility']:.1f}%")
                        
                        if summary_indicators:
                            self.logger.info(f"  {symbol}: {', '.join(summary_indicators)}")
                        else:
                            self.logger.info(f"  {symbol}: Calculating indicators...")
                    else:
                        self.logger.info(f"  {symbol}: No indicators available")
                        
                except Exception as e:
                    self.logger.info(f"  {symbol}: Error getting indicators - {e}")
            
            self.logger.info("="*50)
    
    def run_analysis_cycle(self, symbol: str) -> Dict:
        """
        Run a single analysis cycle for a symbol without trading
        
        Args:
            symbol: Stock symbol to analyse
        """
        try:
            # Get enhanced analysis
            analysis = self.get_enhanced_market_analysis(symbol)
            
            if analysis:
                # Get strategy status
                status = self.strategy.get_strategy_status(symbol)
                
                # Combine analysis with strategy insights
                enhanced_status = {
                    'symbol': symbol,
                    'current_price': analysis['last'],
                    'price_change_pct': analysis.get('price_change_pct', 0),
                    'position': status['position'],
                    'technical_indicators': status['technical_indicators'],
                    'buy_signals': [],
                    'sell_signals': [],
                    'risk_check': {}
                }
                
                # Evaluate signals without executing trades
                if status['technical_indicators']:
                    try:
                        buy_signals = self.strategy.evaluate_buy_signals(
                            symbol, analysis, status['technical_indicators']
                        )
                        sell_signals = self.strategy.evaluate_sell_signals(
                            symbol, analysis, status['technical_indicators']
                        )
                        
                        enhanced_status['buy_signals'] = buy_signals
                        enhanced_status['sell_signals'] = sell_signals
                        enhanced_status['buy_signal_count'] = sum(buy_signals)
                        enhanced_status['sell_signal_count'] = sum(sell_signals)
                        
                        # Risk management check
                        enhanced_status['risk_check'] = self.strategy.check_risk_management(symbol, analysis)
                        
                    except Exception as e:
                        self.logger.error(f"Error evaluating signals for {symbol}: {e}")
                
                return enhanced_status
            else:
                return {'symbol': symbol, 'error': 'No analysis data available'}
                
        except Exception as e:
            self.logger.error(f"Error in analysis cycle for {symbol}: {e}")
            return {'symbol': symbol, 'error': str(e)}
    
    def get_portfolio_summary(self) -> Dict:
        """Get comprehensive portfolio summary"""
        try:
            # Get account information
            account_info = self.ib_conn.get_account_summary()
            
            # Get strategy performance
            strategy_perf = self.strategy.get_performance_summary()
            
            # Calculate total portfolio value
            total_cash = float(account_info.get('AvailableFunds', '0')) if account_info else 0
            total_position_value = strategy_perf.get('total_position_value', 0)
            total_portfolio_value = total_cash + total_position_value
            
            portfolio_summary = {
                'account_info': account_info,
                'strategy_performance': strategy_perf,
                'total_cash': total_cash,
                'total_position_value': total_position_value,
                'total_portfolio_value': total_portfolio_value,
                'positions': self.strategy.positions.copy(),
                'trades_executed': self.trades_executed,
                'session_duration': datetime.now() - self.start_time if self.start_time else None
            }
            
            return portfolio_summary
            
        except Exception as e:
            self.logger.error(f"Error getting portfolio summary: {e}")
            return {'error': str(e)}
    
    def run_diagnostic_check(self) -> Dict:
        """Run comprehensive diagnostic check of all bot components"""
        diagnostics = {
            'connection_status': self.ib_conn.is_connected() if self.ib_conn else False,
            'components_initialised': {
                'historical_data': self.historical_data is not None,
                'market_data': self.market_data is not None,
                'order_manager': self.order_manager is not None,
                'strategy': self.strategy is not None
            },
            'strategy_config': {
                'name': self.strategy.name,
                'short_window': self.strategy.short_window,
                'long_window': self.strategy.long_window,
                'stop_loss_pct': self.strategy.stop_loss_pct * 100,
                'take_profit_pct': self.strategy.take_profit_pct * 100
            },
            'bot_config': {
                'environment': self.environment,
                'symbols': self.symbols,
                'update_interval': self.update_interval,
                'is_running': self.is_running
            },
            'data_availability': {}
        }
        
        # Test data availability for each symbol
        for symbol in self.symbols[:2]:  # Test first 2 symbols to avoid too many requests
            try:
                analysis = self.get_enhanced_market_analysis(symbol)
                diagnostics['data_availability'][symbol] = {
                    'available': analysis is not None,
                    'has_indicators': bool(analysis and analysis.get('technical_indicators')),
                    'price': analysis['last'] if analysis else None
                }
            except Exception as e:
                diagnostics['data_availability'][symbol] = {
                    'available': False,
                    'error': str(e)
                }
        
        return diagnostics

print("Trading Bot class created")

Trading Bot class created


### Trading Bot Simulator

In [26]:
class IBTradingBotSimulator(IBTradingBot):
    """Simulator version of trading bot for testing without real IB connection"""
    
    def __init__(self, strategy: TradingStrategy = None):
        """Initialise enhanced trading bot simulator"""
        self.environment = 'simulator'
        self.logger = logging.getLogger('TradingBot-Simulator')
        
        # Simulated components
        self.ib_conn = None
        self.historical_data = SimulatedHistoricalData()
        self.market_data = SimulatedMarketData()
        self.order_manager = SimulatedOrderManager()
        self.strategy = strategy or TradingStrategy('Enhanced_Simulator', 10, 30)
        
        # Configure strategy with simulated data handlers
        self.strategy.set_data_handlers(self.historical_data, self.market_data)
        
        # Bot configuration
        self.symbols = ['ABCL','ALLO','ANNX','ANRO','ANVS','APLM','APLT','APVO','ARAY','ARBE',
    'ATEN','ATAI','ATHA','AUTL','AVIR','CABA','CAMP','CARM','CATX','CCCC',
    'CELU','CERS','CGTX','CHRS','CLDI','CLLS','CLRB','CLSD','CLYM','CMPS',
    'CMPX','CMTL','COCP','CRBU','CRDF','CRDL','CRIS','CTMX','CTOR','CTXR',
    'CUE','CURR','DARE','DTIL','EDIT','ELDN','EQ','ERAS','ERNA','ESLA',
    'ESPR','EVGN','FATE','FBLG']
        self.is_running = False
        self.update_interval = 5  # Faster for simulation
        
        # Performance tracking
        self.start_time = None
        self.trades_executed = 0
        self.simulated_balance = 10000.0  # Starting balance
        
        self.logger.info("Trading bot simulator initialised")
        self.logger.info(f"Strategy: {self.strategy.name}")
        self.logger.info(f"Starting balance: £{self.simulated_balance:,.2f}")
    
    def connect(self) -> bool:
        """Simulate connection with enhanced features"""
        self.logger.info("Connected to trading simulator")
        return True
    
    def disconnect(self):
        """Simulate disconnection with cleanup"""
        self.is_running = False
        self.logger.info("Disconnected from trading simulator")
    
    def get_enhanced_market_analysis(self, symbol: str) -> Optional[Dict]:
        """
        Get enhanced simulated market analysis
        
        Args:
            symbol: Stock symbol to analyse
        """
        try:
            # Use strategy's enhanced analysis with simulated data
            analysis = self.strategy.get_enhanced_market_analysis(symbol)
            
            if analysis:
                self.logger.info(
                    f"Enhanced simulated analysis for {symbol}: £{analysis['last']:.2f} "
                    f"({analysis.get('price_change_pct', 0):+.2f}%)"
                )
                return analysis
            else:
                # Fallback to basic simulated analysis
                return self.get_market_analysis(symbol)
                
        except Exception as e:
            self.logger.error(f"Error getting enhanced simulated analysis for {symbol}: {e}")
            return self.get_market_analysis(symbol)
    
    def get_market_analysis(self, symbol: str) -> Optional[Dict]:
        """
        Get simulated market analysis data for trading decisions
        
        Args:
            symbol: Stock symbol to analyse
        """
        try:
            # Get simulated historical data
            df = self.historical_data.get_intraday_data(symbol, days=2)
            
            if df is None or df.empty:
                self.logger.warning(f"No simulated historical data available for {symbol}")
                return None
            
            # Calculate current metrics
            latest_price = float(df.iloc[-1]['close'])
            volume = int(df.iloc[-1]['volume'])
            
            # Calculate moving averages if enough data
            if len(df) >= 20:
                ma_20 = df['close'].rolling(window=20).mean().iloc[-1]
            else:
                ma_20 = latest_price
            
            if len(df) >= 50:
                ma_50 = df['close'].rolling(window=50).mean().iloc[-1]
            else:
                ma_50 = latest_price
            
            # Price change from previous close
            if len(df) >= 2:
                price_change = latest_price - float(df.iloc[-2]['close'])
                price_change_pct = (price_change / float(df.iloc[-2]['close'])) * 100
            else:
                price_change = 0
                price_change_pct = 0
            
            analysis_data = {
                'symbol': symbol,
                'last': latest_price,
                'volume': volume,
                'price_change': price_change,
                'price_change_pct': price_change_pct,
                'ma_20': float(ma_20),
                'ma_50': float(ma_50),
                'timestamp': datetime.now(),
                'historical_data': df
            }
            
            self.logger.info(f"Simulated analysis for {symbol}: £{latest_price:.2f} ({price_change_pct:+.2f}%)")
            return analysis_data
            
        except Exception as e:
            self.logger.error(f"Error getting simulated market analysis for {symbol}: {e}")
            return None
    
    def execute_trade_logic(self, symbol: str, market_data: Dict) -> bool:
        """Execute enhanced simulated trading logic"""
        try:
            # Check for buy signal using enhanced strategy
            if self.strategy.should_buy(market_data):
                if self.simulated_balance > 0:
                    quantity = self.strategy.calculate_position_size(
                        symbol, market_data['last'], self.simulated_balance
                    )
                    
                    trade_value = quantity * market_data['last']
                    
                    if trade_value <= self.simulated_balance and quantity > 0:
                        # Execute simulated buy
                        self.order_manager.place_market_order(symbol, 'BUY', quantity)
                        self.strategy.update_position(
                            symbol, 'BUY', quantity, market_data['last']
                        )
                        
                        self.simulated_balance -= trade_value
                        self.trades_executed += 1
                        
                        self.logger.info(
                            f"SIMULATED BUY: {quantity} {symbol} @ £{market_data['last']:.2f} "
                            f"(Cost: £{trade_value:.2f}, Balance: £{self.simulated_balance:.2f})"
                        )
                        return True
            
            # Check for sell signal using enhanced strategy
            elif self.strategy.should_sell(symbol, market_data):
                position = self.strategy.get_position(symbol)
                
                if position['shares'] > 0:
                    # Execute simulated sell
                    self.order_manager.place_market_order(symbol, 'SELL', position['shares'])
                    
                    trade_value = position['shares'] * market_data['last']
                    
                    self.strategy.update_position(
                        symbol, 'SELL', position['shares'], market_data['last']
                    )
                    
                    self.simulated_balance += trade_value
                    self.trades_executed += 1
                    
                    # Calculate P&L
                    cost_basis = position['shares'] * position['avg_price']
                    pnl = trade_value - cost_basis
                    
                    self.logger.info(
                        f"SIMULATED SELL: {position['shares']} {symbol} @ £{market_data['last']:.2f} "
                        f"(Value: £{trade_value:.2f}, P&L: £{pnl:.2f}, Balance: £{self.simulated_balance:.2f})"
                    )
                    return True
            
            return False
            
        except Exception as e:
            self.logger.error(f"Error in simulated trade logic for {symbol}: {e}")
            return False
    
    def run_single_cycle(self):
        """Run a single enhanced simulated trading cycle"""
        trades_this_cycle = 0
        
        for symbol in self.symbols:
            try:
                # Get enhanced simulated market analysis
                market_data = self.get_enhanced_market_analysis(symbol)
                
                if market_data:
                    # Execute enhanced trading logic
                    if self.execute_trade_logic(symbol, market_data):
                        trades_this_cycle += 1
                        
                        # Log enhanced strategy status after trade
                        try:
                            status = self.strategy.get_strategy_status(symbol)
                            if status['technical_indicators']:
                                indicators = status['technical_indicators']
                                self.logger.info(
                                    f"{symbol} indicators - RSI: {indicators.get('rsi', 'N/A'):.1f}, "
                                    f"MACD: {indicators.get('macd', 'N/A'):.3f}, "
                                    f"Volatility: {indicators.get('volatility', 'N/A'):.2f}%"
                                )
                        except Exception as e:
                            self.logger.error(f"Error logging indicators for {symbol}: {e}")
                        
            except Exception as e:
                self.logger.error(f"Error processing {symbol}: {e}")
        
        # Enhanced cycle summary
        if trades_this_cycle > 0:
            self.logger.info(f"Cycle completed: {trades_this_cycle} trades executed")
            
            # Show performance summary after trades
            try:
                perf = self.strategy.get_performance_summary()
                if perf['total_trades'] > 0:
                    self.logger.info(
                        f"Performance - Total Trades: {perf['total_trades']}, "
                        f"Win Rate: {perf['win_rate']:.1f}%, "
                        f"Total P&L: £{perf['total_pnl']:.2f}"
                    )
            except Exception as e:
                self.logger.error(f"Error getting performance summary: {e}")
        
        return True
    
    def run_diagnostic_check(self) -> Dict:
        """Run comprehensive diagnostic check for simulator"""
        diagnostics = {
            'connection_status': True,  # Always true for simulator
            'components_initialised': {
                'historical_data': self.historical_data is not None,
                'market_data': self.market_data is not None,
                'order_manager': self.order_manager is not None,
                'strategy': self.strategy is not None
            },
            'strategy_config': {
                'name': self.strategy.name,
                'short_window': self.strategy.short_window,
                'long_window': self.strategy.long_window,
                'stop_loss_pct': self.strategy.stop_loss_pct * 100,
                'take_profit_pct': self.strategy.take_profit_pct * 100
            },
            'bot_config': {
                'environment': self.environment,
                'symbols': self.symbols,
                'update_interval': self.update_interval,
                'is_running': self.is_running,
                'simulated_balance': self.simulated_balance
            },
            'data_availability': {}
        }
        
        # Test data availability for each symbol
        for symbol in self.symbols[:2]:  # Test first 2 symbols
            try:
                analysis = self.get_enhanced_market_analysis(symbol)
                diagnostics['data_availability'][symbol] = {
                    'available': analysis is not None,
                    'has_indicators': bool(analysis and analysis.get('technical_indicators')),
                    'price': analysis['last'] if analysis else None
                }
            except Exception as e:
                diagnostics['data_availability'][symbol] = {
                    'available': False,
                    'error': str(e)
                }
        
        return diagnostics
    
    def get_portfolio_summary(self) -> Dict:
        """Get comprehensive simulated portfolio summary"""
        try:
            # Get strategy performance
            strategy_perf = self.strategy.get_performance_summary()
            
            # Calculate total portfolio value
            total_cash = self.simulated_balance
            total_position_value = strategy_perf.get('total_position_value', 0)
            total_portfolio_value = total_cash + total_position_value
            
            portfolio_summary = {
                'account_info': {'AvailableFunds': str(self.simulated_balance)},
                'strategy_performance': strategy_perf,
                'total_cash': total_cash,
                'total_position_value': total_position_value,
                'total_portfolio_value': total_portfolio_value,
                'positions': self.strategy.positions.copy(),
                'trades_executed': self.trades_executed,
                'session_duration': datetime.now() - self.start_time if self.start_time else None
            }
            
            return portfolio_summary
            
        except Exception as e:
            self.logger.error(f"Error getting simulated portfolio summary: {e}")
            return {'error': str(e)}


class SimulatedMarketData:
    """Simulated market data handler for testing"""
    
    def __init__(self):
        """Initialise simulated market data handler"""
        self.logger = logging.getLogger('SimulatedMarketData')
        self.subscribed_tickers = {}
        self.last_prices = {}
        
        # Base prices for simulation
        self.base_prices = {
            'MRK': 180.0, 'UNH': 850.0, 'LLY': 380.0, 'PG': 160.0, 'KO': 65.0,
            'WMT': 175.0, 'TSLA': 250.0, 'NVDA': 800.0, 'AMZN': 180.0, 
            'GOOGL': 140.0, 'META': 520.0, 'QCOM': 170.0
        }
    
    def get_market_data(self, symbol: str, exchange: str = 'SMART',
                       currency: str = 'USD') -> Optional[Dict]:
        """
        Get simulated real-time market data
        
        Args:
            symbol: Stock symbol
            exchange: Exchange (ignored in simulation)
            currency: Currency (ignored in simulation)
        """
        try:
            if symbol not in self.base_prices:
                self.base_prices[symbol] = random.uniform(50.0, 500.0)
            
            base_price = self.base_prices[symbol]
            
            # Generate realistic market data
            volatility = random.gauss(0, 0.005)  # 0.5% volatility
            current_price = base_price * (1 + volatility)
            
            # Generate bid/ask spread
            spread_pct = random.uniform(0.001, 0.003)  # 0.1-0.3% spread
            spread = current_price * spread_pct
            
            market_data = {
                'symbol': symbol,
                'last': round(current_price, 2),
                'bid': round(current_price - spread/2, 2),
                'ask': round(current_price + spread/2, 2),
                'volume': random.randint(1000, 50000),
                'high': round(current_price * 1.01, 2),
                'low': round(current_price * 0.99, 2),
                'close': round(current_price, 2),
                'timestamp': datetime.now()
            }
            
            # Update base price for next call
            self.base_prices[symbol] = current_price
            self.last_prices[symbol] = current_price
            
            return market_data
            
        except Exception as e:
            self.logger.error(f"Error getting simulated market data for {symbol}: {e}")
            return None
    
    def get_market_snapshot(self, symbol: str, exchange: str = 'SMART',
                           currency: str = 'USD') -> Optional[Dict]:
        """Get simulated market snapshot"""
        return self.get_market_data(symbol, exchange, currency)
    
    def subscribe_to_ticker(self, symbol: str, exchange: str = 'SMART',
                           currency: str = 'USD') -> bool:
        """Simulate ticker subscription"""
        self.subscribed_tickers[symbol] = {'symbol': symbol}
        self.logger.info(f"Subscribed to simulated {symbol}")
        return True
    
    def unsubscribe_from_ticker(self, symbol: str) -> bool:
        """Simulate ticker unsubscription"""
        if symbol in self.subscribed_tickers:
            del self.subscribed_tickers[symbol]
            self.logger.info(f"Unsubscribed from simulated {symbol}")
            return True
        return False
    
    def get_subscribed_data(self, symbol: str) -> Optional[Dict]:
        """Get simulated subscribed data"""
        if symbol in self.subscribed_tickers:
            return self.get_market_data(symbol)
        return None
    
    def cleanup_subscriptions(self):
        """Cancel all simulated subscriptions"""
        for symbol in list(self.subscribed_tickers.keys()):
            self.unsubscribe_from_ticker(symbol)
        self.logger.info("All simulated market data subscriptions cancelled")
    
    def get_last_known_price(self, symbol: str) -> Optional[float]:
        """Get last known simulated price"""
        return self.last_prices.get(symbol)
    
    def is_market_open(self) -> bool:
        """Simulate market hours (always open for testing)"""
        return True


class SimulatedHistoricalData:
    """Enhanced simulated historical data generator for testing"""
    
    def __init__(self):
        """Initialise with realistic starting prices and enhanced data history"""
        self.base_prices = {
            'MRK': 180.0, 'UNH': 850.0, 'LLY': 380.0, 'PG': 160.0, 'KO': 65.0,
            'WMT': 175.0, 'TSLA': 250.0, 'NVDA': 800.0, 'AMZN': 180.0, 
            'GOOGL': 140.0, 'META': 520.0, 'QCOM': 170.0
        }
        self.volatility = 0.002  # 0.2% volatility per minute
        self.trend = 0.0001  # Small upward trend
        self.data_cache = {}
        self.time_counter = {}
        self.logger = logging.getLogger('SimulatedHistoricalData')
    
    def generate_enhanced_historical_data(self, symbol: str, periods: int) -> pd.DataFrame:
        """
        Generate realistic historical price data with enhanced features
        
        Args:
            symbol: Stock symbol
            periods: Number of periods to generate
        """
        if symbol not in self.base_prices:
            self.base_prices[symbol] = random.uniform(50.0, 500.0)
        
        base_price = self.base_prices[symbol]
        prices = []
        
        current_price = base_price
        start_time = datetime.now() - timedelta(minutes=periods)
        
        # Add some market regime changes for more realistic testing
        regime_changes = random.choices([True, False], weights=[0.1, 0.9], k=periods)
        trend_strength = 1.0
        
        for i in range(periods):
            # Simulate market regime changes
            if regime_changes[i]:
                trend_strength *= random.uniform(0.5, 1.5)
                trend_strength = max(0.2, min(2.0, trend_strength))  # Keep within bounds
            
            # Generate realistic price movement with enhanced volatility patterns
            trend_component = self.trend * trend_strength * random.uniform(0.5, 1.5)
            
            # Add volatility clustering (periods of high/low volatility)
            vol_multiplier = random.uniform(0.5, 2.0) if i % 50 == 0 else random.uniform(0.8, 1.2)
            volatility_component = random.gauss(0, self.volatility * vol_multiplier)
            
            price_change = trend_component + volatility_component
            current_price = current_price * (1 + price_change)
            
            # Ensure reasonable price bounds with some drift allowance
            min_price = base_price * 0.7  # Allow 30% down
            max_price = base_price * 1.4  # Allow 40% up
            current_price = max(current_price, min_price)
            current_price = min(current_price, max_price)
            
            # Generate realistic OHLC data with proper relationships
            daily_range = current_price * random.uniform(0.005, 0.03)  # 0.5-3% daily range
            
            open_price = current_price * random.uniform(0.995, 1.005)
            high = current_price + (daily_range * random.uniform(0.3, 1.0))
            low = current_price - (daily_range * random.uniform(0.3, 1.0))
            close_price = current_price
            
            # Ensure OHLC relationships are valid
            high = max(high, open_price, close_price)
            low = min(low, open_price, close_price)
            
            # Generate realistic volume with patterns
            base_volume = random.randint(5000, 50000)
            volume_multiplier = 2.0 if abs(price_change) > 0.01 else 1.0  # Higher volume on big moves
            volume = int(base_volume * volume_multiplier * random.uniform(0.7, 1.3))
            
            prices.append({
                'date': start_time + timedelta(minutes=i),
                'open': round(open_price, 2),
                'high': round(high, 2),
                'low': round(low, 2),
                'close': round(close_price, 2),
                'volume': volume
            })
        
        # Update current price for this symbol
        self.base_prices[symbol] = current_price
        
        df = pd.DataFrame(prices)
        df['timestamp'] = df['date']
        df['symbol'] = symbol
        
        return df
    
    def get_intraday_data(self, symbol: str, days: int = 1) -> Optional[pd.DataFrame]:
        """
        Get enhanced simulated intraday data
        
        Args:
            symbol: Stock symbol
            days: Number of days
        """
        periods = days * 390  # Approximate trading minutes per day
        return self.generate_enhanced_historical_data(symbol, periods)
    
    def get_daily_data(self, symbol: str, period: str = '1 M') -> Optional[pd.DataFrame]:
        """
        Get enhanced simulated daily data
        
        Args:
            symbol: Stock symbol
            period: Time period
        """
        # Convert period to number of days
        period_map = {'1 M': 30, '3 M': 90, '6 M': 180, '1 Y': 365, '2 Y': 730}
        days = period_map.get(period, 30)
        
        return self.generate_enhanced_historical_data(symbol, days)
    
    def get_historical_data(self, symbol: str, duration: str = '1 D',
                           bar_size: str = '1 min', exchange: str = 'SMART',
                           currency: str = 'USD') -> Optional[pd.DataFrame]:
        """
        Get enhanced simulated historical data with IB-like parameters
        
        Args:
            symbol: Stock symbol
            duration: Data duration
            bar_size: Bar size
            exchange: Exchange (ignored in simulation)
            currency: Currency (ignored in simulation)
        """
        # Parse duration to get number of periods
        try:
            if 'D' in duration:
                days = int(duration.replace(' D', ''))
                if '1 min' in bar_size:
                    periods = days * 390  # Trading minutes per day
                elif '1 hour' in bar_size:
                    periods = int(days * 6.5)  # Trading hours per day
                else:  # Daily bars
                    periods = days
            elif 'W' in duration:
                weeks = int(duration.replace(' W', ''))
                periods = weeks * 5  # Trading days per week
            elif 'M' in duration:
                months = int(duration.replace(' M', ''))
                periods = months * 22  # Trading days per month
            else:
                periods = 390  # Default to 1 day
            
            return self.generate_enhanced_historical_data(symbol, int(periods))
            
        except Exception as e:
            self.logger.error(f"Error parsing duration '{duration}': {e}")
            return self.generate_enhanced_historical_data(symbol, 390)
    
    def get_cached_data(self, symbol: str, duration: str = '1 D',
                       bar_size: str = '1 min', max_age_minutes: int = 5) -> Optional[pd.DataFrame]:
        """
        Simulate enhanced cached data behaviour
        """
        # In simulation, always return fresh data with some caching simulation
        cache_key = f"{symbol}_{duration}_{bar_size}"
        
        if cache_key in self.data_cache:
            cached_item = self.data_cache[cache_key]
            age_minutes = (datetime.now() - cached_item['retrieved_at']).total_seconds() / 60
            
            if age_minutes <= max_age_minutes:
                self.logger.info(f"Using simulated cached data for {symbol} (age: {age_minutes:.1f}min)")
                return cached_item['data']
        
        # Generate fresh data and cache it
        fresh_data = self.get_historical_data(symbol, duration, bar_size)
        if fresh_data is not None:
            self.data_cache[cache_key] = {
                'data': fresh_data,
                'retrieved_at': datetime.now()
            }
        
        return fresh_data
    
    def clear_cache(self):
        """Clear simulated cache"""
        self.data_cache.clear()
        self.logger.info("Simulated data cache cleared")
    
    def get_multiple_symbols(self, symbols: List[str], duration: str = '1 D',
                            bar_size: str = '1 min') -> Dict[str, pd.DataFrame]:
        """
        Get enhanced simulated data for multiple symbols
        
        Args:
            symbols: List of stock symbols
            duration: Data duration
            bar_size: Bar size
        """
        results = {}
        
        for symbol in symbols:
            self.logger.info(f"Fetching simulated data for {symbol}")
            data = self.get_historical_data(symbol, duration, bar_size)
            
            if data is not None:
                results[symbol] = data
            
            # Simulate small delay
            time.sleep(0.05)
        
        self.logger.info(f"Retrieved simulated data for {len(results)} out of {len(symbols)} symbols")
        return results


class SimulatedOrderManager:
    """Enhanced simulated order manager for testing"""
    
    def __init__(self):
        """Initialise enhanced simulated order manager"""
        self.orders_log = []
        self.order_id_counter = 1000
        self.logger = logging.getLogger('SimulatedOrderManager')
        self.open_orders = []
    
    def place_market_order(self, symbol: str, action: str, quantity: int,
                          exchange: str = 'SMART', currency: str = 'USD') -> bool:
        """Simulate placing a market order with enhanced logging"""
        order = {
            'order_id': self.order_id_counter,
            'symbol': symbol,
            'action': action,
            'quantity': quantity,
            'order_type': 'MARKET',
            'status': 'FILLED',
            'timestamp': datetime.now(),
            'exchange': exchange,
            'currency': currency
        }
        
        self.orders_log.append(order)
        self.order_id_counter += 1
        
        self.logger.info(f"Simulated {action} order placed: {quantity} {symbol}")
        return True
    
    def place_limit_order(self, symbol: str, action: str, quantity: int,
                         limit_price: float, exchange: str = 'SMART',
                         currency: str = 'USD') -> bool:
        """Simulate placing a limit order"""
        order = {
            'order_id': self.order_id_counter,
            'symbol': symbol,
            'action': action,
            'quantity': quantity,
            'order_type': 'LIMIT',
            'limit_price': limit_price,
            'status': 'SUBMITTED',
            'timestamp': datetime.now(),
            'exchange': exchange,
            'currency': currency
        }
        
        self.orders_log.append(order)
        self.open_orders.append(order)
        self.order_id_counter += 1
        
        self.logger.info(f"Simulated limit order placed: {action} {quantity} {symbol} @ £{limit_price:.2f}")
        return True
    
    def get_open_orders(self) -> List[Dict]:
        """Get simulated open orders"""
        return self.open_orders.copy()
    
    def cancel_order(self, order_id: int) -> bool:
        """Simulate order cancellation"""
        for order in self.open_orders:
            if order['order_id'] == order_id:
                order['status'] = 'CANCELLED'
                self.open_orders.remove(order)
                self.logger.info(f"Simulated order {order_id} cancelled")
                return True
        return False
    
    def get_orders_log(self) -> List[Dict]:
        """Get enhanced simulated orders log"""
        return self.orders_log


print("Trading Bot Simulator created")

Trading Bot Simulator created


### Configuration and Testing Functions

In [27]:
def create_trading_bot(environment: str = 'paper', 
                      strategy_type: str = 'enhanced',
                      **strategy_params) -> IBTradingBot:
    """
    Create a trading bot with specified configuration
    
    Args:
        environment: 'paper', 'live', or 'simulator'
        strategy_type: 'enhanced' for Enhanced Trading Strategy, 'sma' for legacy
        **strategy_params: Strategy-specific parameters
    """
    
    # Create enhanced strategy with proper configuration
    if strategy_type.lower() == 'enhanced':
        short_window = strategy_params.get('short_window', 10)
        long_window = strategy_params.get('long_window', 30)
        strategy_name = strategy_params.get('name', f'Enhanced_{short_window}_{long_window}')
        strategy = TradingStrategy(strategy_name, short_window, long_window)
        
        # Configure risk management parameters
        strategy.stop_loss_pct = strategy_params.get('stop_loss_pct', 0.05)
        strategy.take_profit_pct = strategy_params.get('take_profit_pct', 0.15)
        strategy.max_position_size = strategy_params.get('max_position_size', 0.20)
        
    elif strategy_type.lower() == 'sma':
        # Legacy simple strategy for backwards compatibility
        short_window = strategy_params.get('short_window', 10)
        long_window = strategy_params.get('long_window', 30)
        strategy_name = f'SMA_{short_window}_{long_window}'
        strategy = TradingStrategy(strategy_name, short_window, long_window)
    else:
        # Default to enhanced strategy
        strategy = TradingStrategy('Enhanced_Default', 10, 30)
    
    # Create appropriate bot
    if environment.lower() == 'simulator':
        bot = IBTradingBotSimulator(strategy)
    else:
        bot = IBTradingBot(environment, strategy)
    
    return bot


def test_bot_connection(environment: str = 'paper') -> bool:
    """
    Test bot connection with enhanced diagnostics
    
    Args:
        environment: 'paper', 'live', or 'simulator'
    """
    print(f"Testing {environment} trading connection...")
    
    if environment == 'simulator':
        bot = create_trading_bot('simulator')
        success = bot.connect()
        if success:
            print("Simulator connection test passed")
            
            # Test enhanced strategy features
            print("\nTesting enhanced strategy features:")
            for symbol in ['MRK', 'UNH'][:1]:  # Test one symbol
                try:
                    analysis = bot.get_enhanced_market_analysis(symbol)
                    if analysis and analysis.get('technical_indicators'):
                        indicators = analysis['technical_indicators']
                        print(f"  {symbol} Indicators:")
                        print(f"    RSI: {indicators.get('rsi', 'N/A'):.1f}")
                        print(f"    MACD: {indicators.get('macd', 'N/A'):.3f}")
                        print(f"    Volatility: {indicators.get('volatility', 'N/A'):.1f}%")
                        print(f"    Bollinger Band Width: {indicators.get('bb_width', 'N/A'):.1f}%")
                    else:
                        print(f"  {symbol}: Basic analysis available")
                except Exception as e:
                    print(f"  {symbol}: Error testing indicators - {e}")
        
        bot.disconnect()
        return success
    
    bot = create_trading_bot(environment)
    
    try:
        success = bot.connect()
        
        if success:
            print(f"Connection to {environment} trading successful")
            
            # Test enhanced historical data and analysis
            print("\nTesting enhanced analysis capabilities...")
            test_symbols = ['MRK', 'UNH']
            
            for symbol in test_symbols:
                try:
                    # Test enhanced market analysis
                    analysis = bot.get_enhanced_market_analysis(symbol)
                    if analysis:
                        print(f"  {symbol}: £{analysis['last']:.2f}")
                        
                        # Show technical indicators if available
                        if analysis.get('technical_indicators'):
                            indicators = analysis['technical_indicators']
                            indicator_summary = []
                            
                            if 'rsi' in indicators:
                                indicator_summary.append(f"RSI: {indicators['rsi']:.1f}")
                            if 'macd' in indicators:
                                indicator_summary.append(f"MACD: {indicators['macd']:.3f}")
                            if 'volatility' in indicators:
                                indicator_summary.append(f"Vol: {indicators['volatility']:.1f}%")
                            
                            if indicator_summary:
                                print(f"    Indicators: {', '.join(indicator_summary)}")
                            
                        # Test signal evaluation
                        if analysis.get('technical_indicators'):
                            try:
                                buy_signals = bot.strategy.evaluate_buy_signals(
                                    symbol, analysis, analysis['technical_indicators']
                                )
                                sell_signals = bot.strategy.evaluate_sell_signals(
                                    symbol, analysis, analysis['technical_indicators']
                                )
                                
                                print(f"    Buy Signals: {sum(buy_signals)}/{len(buy_signals)}")
                                print(f"    Sell Signals: {sum(sell_signals)}/{len(sell_signals)}")
                                
                            except Exception as e:
                                print(f"    Signal evaluation error: {e}")
                    else:
                        print(f"  {symbol}: No enhanced analysis available")
                        
                except Exception as e:
                    print(f"  {symbol}: Error getting enhanced analysis - {e}")
            
            # Test account info with enhanced display
            print("\nAccount Information:")
            try:
                account_info = bot.ib_conn.get_account_summary()
                if account_info:
                    net_liq = account_info.get('NetLiquidation', 'N/A')
                    buying_power = account_info.get('BuyingPower', 'N/A')
                    available_funds = account_info.get('AvailableFunds', 'N/A')
                    print(f"  Net Liquidation: £{net_liq}")
                    print(f"  Buying Power: £{buying_power}")
                    print(f"  Available Funds: £{available_funds}")
                else:
                    print("  Account info not available")
            except Exception as e:
                print(f"  Error getting account info: {e}")
            
            # Run diagnostic check
            print("\nDiagnostic Check:")
            try:
                diagnostics = bot.run_diagnostic_check()
                print(f"  Connection: {'Connected' if diagnostics['connection_status'] else 'Disconnected'}")
                print(f"  Components: {sum(diagnostics['components_initialised'].values())}/4 initialised")
                print(f"  Strategy: {diagnostics['strategy_config']['name']}")
                print(f"  Risk Management: Stop Loss {diagnostics['strategy_config']['stop_loss_pct']:.1f}%, Take Profit {diagnostics['strategy_config']['take_profit_pct']:.1f}%")
            except Exception as e:
                print(f"  Diagnostic error: {e}")
            
        else:
            print(f"Failed to connect to {environment} trading")
            if environment == 'paper':
                print("\nTroubleshooting tips:")
                print("• Ensure TWS or IB Gateway is running")
                print("• Check it's configured for paper trading (port 7497)")
                print("• Enable API connections: Configure > API > Enable ActiveX and Socket Clients")
                print("• Check if 'Read-Only API' is disabled")
                print("• Verify no other API clients are connected with the same client ID")
                print("• Check your IP address is whitelisted in TWS API settings")
            
    except Exception as e:
        print(f"Connection test failed: {e}")
        success = False
    finally:
        try:
            bot.disconnect()
        except:
            pass
    
    return success


def run_strategy_backtest(symbols: List[str] = None, 
                         duration_minutes: int = 5,
                         strategy_config: Dict = None) -> Dict:
    """
    Run enhanced backtest using the simulator with configurable strategy
    
    Args:
        symbols: List of symbols to test
        duration_minutes: How long to run the test
        strategy_config: Strategy configuration parameters
    """
    print("Running enhanced strategy backtest simulation...")
    
    # Configure strategy
    config = strategy_config or {}
    
    # Create simulator bot with enhanced strategy
    bot = create_trading_bot(
        'simulator',
        strategy_type='enhanced',
        short_window=config.get('short_window', 10),
        long_window=config.get('long_window', 30),
        stop_loss_pct=config.get('stop_loss_pct', 0.05),
        take_profit_pct=config.get('take_profit_pct', 0.15),
        name=config.get('name', 'Backtest_Enhanced')
    )
    
    if symbols:
        bot.set_symbols(symbols)
    
    # Connect and run
    if bot.connect():
        # Calculate max cycles based on duration
        max_cycles = (duration_minutes * 60) // bot.update_interval
        
        print(f"Running {max_cycles} cycles over {duration_minutes} minutes...")
        print(f"Testing symbols: {bot.symbols}")
        print(f"Strategy: {bot.strategy.name}")
        print(f"Starting balance: £{bot.simulated_balance:,.2f}")
        print(f"Risk Management: Stop Loss {bot.strategy.stop_loss_pct*100:.1f}%, Take Profit {bot.strategy.take_profit_pct*100:.1f}%")
        
        bot.start_trading(max_cycles=max_cycles)
        
        # Calculate enhanced portfolio metrics
        portfolio_value = bot.simulated_balance
        total_unrealised_pnl = 0.0
        
        for symbol, position in bot.strategy.positions.items():
            if position['shares'] > 0:
                try:
                    analysis = bot.get_enhanced_market_analysis(symbol)
                    if analysis:
                        current_value = position['shares'] * analysis['last']
                        cost_basis = position['shares'] * position['avg_price']
                        unrealised_pnl = current_value - cost_basis
                        
                        portfolio_value += current_value
                        total_unrealised_pnl += unrealised_pnl
                except:
                    # Fallback to cost basis if current price unavailable
                    portfolio_value += position['shares'] * position['avg_price']
        
        # Get enhanced results
        strategy_performance = bot.strategy.get_performance_summary()
        
        results = {
            'total_trades': bot.trades_executed,
            'final_balance': bot.simulated_balance,
            'portfolio_value': portfolio_value,
            'total_return': portfolio_value - 10000.0,
            'return_percentage': ((portfolio_value - 10000.0) / 10000.0) * 100,
            'total_unrealised_pnl': total_unrealised_pnl,
            'strategy_performance': strategy_performance,
            'positions': bot.strategy.positions.copy(),
            'orders_log': bot.order_manager.get_orders_log(),
            'technical_indicators': bot.strategy.technical_indicators.copy(),
            'signal_history': {k: v[-10:] for k, v in bot.strategy.signal_history.items()}  # Last 10 signals per symbol
        }
        
        # Display enhanced results
        print(f"\nEnhanced Backtest Results:")
        print(f"Total Trades: {results['total_trades']}")
        print(f"Cash Balance: £{results['final_balance']:.2f}")
        print(f"Portfolio Value: £{results['portfolio_value']:.2f}")
        print(f"Total Return: £{results['total_return']:.2f} ({results['return_percentage']:+.2f}%)")
        
        if results['strategy_performance']['total_trades'] > 0:
            perf = results['strategy_performance']
            print(f"Win Rate: {perf['win_rate']:.1f}%")
            print(f"Profitable Trades: {perf['profitable_trades']}/{perf['total_trades']}")
            print(f"Realised P&L: £{perf['total_pnl']:.2f}")
        
        if total_unrealised_pnl != 0:
            print(f"Unrealised P&L: £{total_unrealised_pnl:+.2f}")
        
        if results['total_trades'] > 0:
            print(f"\nFinal Positions:")
            for symbol, position in results['positions'].items():
                if position['shares'] > 0:
                    cost_basis = position['shares'] * position['avg_price']
                    print(f"  {symbol}: {position['shares']} shares @ £{position['avg_price']:.2f} (Cost: £{cost_basis:.2f})")
        
        # Show technical analysis summary
        print(f"\nTechnical Analysis Summary:")
        for symbol, indicators in results['technical_indicators'].items():
            if indicators:
                summary_parts = []
                if 'rsi' in indicators:
                    rsi_status = "Oversold" if indicators['rsi'] < 30 else "Overbought" if indicators['rsi'] > 70 else "Neutral"
                    summary_parts.append(f"RSI: {indicators['rsi']:.1f} ({rsi_status})")
                if 'macd' in indicators and 'macd_signal' in indicators:
                    macd_status = "Bullish" if indicators['macd'] > indicators['macd_signal'] else "Bearish"
                    summary_parts.append(f"MACD: {macd_status}")
                if 'volatility' in indicators:
                    vol_status = "High" if indicators['volatility'] > 3.0 else "Low" if indicators['volatility'] < 1.0 else "Normal"
                    summary_parts.append(f"Volatility: {vol_status} ({indicators['volatility']:.1f}%)")
                
                if summary_parts:
                    print(f"  {symbol}: {', '.join(summary_parts)}")
                else:
                    print(f"  {symbol}: Basic indicators calculated")
        
        bot.disconnect()
        return results
    else:
        print("Failed to connect to simulator")
        return {}


def test_historical_data(symbol: str = 'MRK', environment: str = 'paper'):
    """
    Test enhanced historical data retrieval and analysis
    
    Args:
        symbol: Symbol to test
        environment: Trading environment
    """
    print(f"Testing enhanced historical data for {symbol} in {environment} environment...")
    
    bot = create_trading_bot(environment, strategy_type='enhanced')
    
    if bot.connect():
        try:
            # Test enhanced analysis capabilities
            print(f"\n1. Enhanced Market Analysis:")
            analysis = bot.get_enhanced_market_analysis(symbol)
            if analysis:
                print(f"   Current price: £{analysis['last']:.2f}")
                print(f"   Price change: {analysis.get('price_change_pct', 0):+.2f}%")
                
                # Display technical indicators
                if analysis.get('technical_indicators'):
                    indicators = analysis['technical_indicators']
                    print(f"   Technical Indicators:")
                    
                    if 'rsi' in indicators:
                        rsi_level = "Oversold" if indicators['rsi'] < 30 else "Overbought" if indicators['rsi'] > 70 else "Neutral"
                        print(f"     RSI: {indicators['rsi']:.1f} ({rsi_level})")
                    
                    if 'macd' in indicators and 'macd_signal' in indicators:
                        macd_trend = "Bullish" if indicators['macd'] > indicators['macd_signal'] else "Bearish"
                        print(f"     MACD: {indicators['macd']:.3f} vs Signal: {indicators['macd_signal']:.3f} ({macd_trend})")
                    
                    if 'bb_upper' in indicators and 'bb_lower' in indicators:
                        bb_position = "Above Upper" if analysis['last'] > indicators['bb_upper'] else \
                                     "Below Lower" if analysis['last'] < indicators['bb_lower'] else "Within Bands"
                        print(f"     Bollinger Bands: £{indicators['bb_lower']:.2f} - £{indicators['bb_upper']:.2f} ({bb_position})")
                    
                    if 'volatility' in indicators:
                        vol_level = "High" if indicators['volatility'] > 3.0 else "Low" if indicators['volatility'] < 1.0 else "Normal"
                        print(f"     Volatility: {indicators['volatility']:.2f}% ({vol_level})")
                    
                    if 'volume_ratio' in indicators:
                        vol_activity = "High" if indicators['volume_ratio'] > 1.5 else "Low" if indicators['volume_ratio'] < 0.7 else "Normal"
                        print(f"     Volume Activity: {indicators['volume_ratio']:.1f}x average ({vol_activity})")
                    
                    # Test signal evaluation
                    print(f"   Signal Evaluation:")
                    try:
                        buy_signals = bot.strategy.evaluate_buy_signals(symbol, analysis, indicators)
                        sell_signals = bot.strategy.evaluate_sell_signals(symbol, analysis, indicators)
                        
                        print(f"     Buy Signals: {sum(buy_signals)}/{len(buy_signals)} active")
                        print(f"     Sell Signals: {sum(sell_signals)}/{len(sell_signals)} active")
                        
                        # Show which specific signals are active
                        if any(buy_signals):
                            active_buy = [i for i, signal in enumerate(buy_signals) if signal]
                            print(f"     Active Buy Signal Types: {active_buy}")
                        
                        if any(sell_signals):
                            active_sell = [i for i, signal in enumerate(sell_signals) if signal]
                            print(f"     Active Sell Signal Types: {active_sell}")
                            
                    except Exception as e:
                        print(f"     Error evaluating signals: {e}")
                
                # Test daily trend analysis
                if analysis.get('daily_trend'):
                    trend = analysis['daily_trend']
                    print(f"   Daily Trend: {trend.get('direction', 'Unknown')} (Strength: {trend.get('strength', 0):.1f}%)")
                    
                    if 'momentum_5d' in trend:
                        print(f"   5-Day Momentum: {trend['momentum_5d']:+.2f}%")
                    if 'momentum_20d' in trend:
                        print(f"   20-Day Momentum: {trend['momentum_20d']:+.2f}%")
                        
            else:
                print(f"   No enhanced analysis data available")
            
            # Test multiple data timeframes
            print(f"\n2. Historical Data Timeframes:")
            
            # Test intraday data
            intraday = bot.historical_data.get_intraday_data(symbol, days=1)
            if intraday is not None and not intraday.empty:
                print(f"   Intraday (1 day): {len(intraday)} data points")
                print(f"   Latest price: £{intraday.iloc[-1]['close']:.2f}")
                print(f"   Date range: {intraday.iloc[0]['date']} to {intraday.iloc[-1]['date']}")
            else:
                print(f"   Intraday: No data available")
            
            # Test daily data
            daily = bot.historical_data.get_daily_data(symbol, period='1 M')
            if daily is not None and not daily.empty:
                print(f"   Daily (1 month): {len(daily)} data points")
                monthly_return = ((daily.iloc[-1]['close'] - daily.iloc[0]['close']) / daily.iloc[0]['close']) * 100
                print(f"   Monthly return: {monthly_return:+.2f}%")
            else:
                print(f"   Daily: No data available")
            
            # Test strategy status
            print(f"\n3. Strategy Status:")
            try:
                status = bot.strategy.get_strategy_status(symbol)
                print(f"   Position: {status['position']['shares']} shares")
                print(f"   Price History Length: {status['price_history_length']}")
                print(f"   Technical Indicators Count: {len(status['technical_indicators'])}")
                print(f"   Recent Signals: {len(status['recent_signals'])}")
            except Exception as e:
                print(f"   Error getting strategy status: {e}")
                
        except Exception as e:
            print(f"Error testing enhanced historical data: {e}")
        finally:
            bot.disconnect()
    else:
        print(f"Failed to connect to {environment} environment")


def run_enhanced_analysis_test(symbols: List[str] = None) -> Dict:
    """
    Run comprehensive analysis test without trading
    
    Args:
        symbols: List of symbols to analyse
    """
    symbols = symbols or ['MRK', 'UNH', 'LLY']
    print(f"Running enhanced analysis test for: {symbols}")
    
    bot = create_trading_bot('simulator', strategy_type='enhanced')
    
    analysis_results = {}
    
    if bot.connect():
        try:
            for symbol in symbols:
                print(f"\n--- Analysing {symbol} ---")
                
                # Run analysis cycle (no trading)
                cycle_result = bot.run_analysis_cycle(symbol)
                analysis_results[symbol] = cycle_result
                
                if 'error' not in cycle_result:
                    print(f"Current Price: £{cycle_result['current_price']:.2f}")
                    print(f"Price Change: {cycle_result['price_change_pct']:+.2f}%")
                    print(f"Position: {cycle_result['position']['shares']} shares")
                    
                    # Technical indicators summary
                    if cycle_result['technical_indicators']:
                        indicators = cycle_result['technical_indicators']
                        print(f"Technical Analysis:")
                        
                        for indicator_name, value in indicators.items():
                            if isinstance(value, (int, float)):
                                if indicator_name == 'rsi':
                                    level = "Oversold" if value < 30 else "Overbought" if value > 70 else "Neutral"
                                    print(f"  {indicator_name.upper()}: {value:.1f} ({level})")
                                elif 'volatility' in indicator_name.lower():
                                    level = "High" if value > 3.0 else "Low" if value < 1.0 else "Normal"
                                    print(f"  {indicator_name}: {value:.2f}% ({level})")
                                elif 'volume' in indicator_name.lower() and 'ratio' in indicator_name.lower():
                                    level = "High" if value > 1.5 else "Low" if value < 0.7 else "Normal"
                                    print(f"  Volume Activity: {value:.1f}x ({level})")
                    
                    # Signal analysis
                    if cycle_result['buy_signals'] and cycle_result['sell_signals']:
                        print(f"Signal Analysis:")
                        print(f"  Buy Signals: {cycle_result['buy_signal_count']}/{len(cycle_result['buy_signals'])}")
                        print(f"  Sell Signals: {cycle_result['sell_signal_count']}/{len(cycle_result['sell_signals'])}")
                        
                        # Determine overall signal
                        if cycle_result['buy_signal_count'] >= len(cycle_result['buy_signals']) // 2:
                            print(f"  Overall Signal: BUY (Strength: {cycle_result['buy_signal_count']}/{len(cycle_result['buy_signals'])})")
                        elif cycle_result['sell_signal_count'] >= len(cycle_result['sell_signals']) // 2:
                            print(f"  Overall Signal: SELL (Strength: {cycle_result['sell_signal_count']}/{len(cycle_result['sell_signals'])})")
                        else:
                            print(f"  Overall Signal: HOLD")
                    
                    # Risk assessment
                    if cycle_result['risk_check']:
                        risk = cycle_result['risk_check']
                        if risk.get('current_pnl_pct'):
                            print(f"  Current P&L: {risk['current_pnl_pct']:+.2f}%")
                        if risk.get('stop_loss'):
                            print(f"  Risk Alert: Stop Loss Triggered")
                        if risk.get('take_profit'):
                            print(f"  Profit Alert: Take Profit Triggered")
                
                else:
                    print(f"Analysis Error: {cycle_result['error']}")
                    
        except Exception as e:
            print(f"Error in enhanced analysis test: {e}")
        finally:
            bot.disconnect()
    else:
        print("Failed to connect to simulator")
    
    return analysis_results


def display_trading_help():
    """Display comprehensive help information for using the enhanced trading bot"""
    help_text = """
    ╔═══════════════════════════════════════════════════════════════════════════════════════╗
                          ENHANCED INTERACTIVE BROKERS TRADING BOT GUIDE
    ╚═══════════════════════════════════════════════════════════════════════════════════════╝

    ENVIRONMENTS:
    • 'paper'     - Paper trading (demo account, port 7497)
    • 'live'      - Live trading (real money, port 7496) 
    • 'simulator' - Enhanced simulator for testing (no IB connection needed)

    ENHANCED QUICK START:

    1. Test Enhanced Connection:
       test_bot_connection('paper')     # Test paper trading with enhanced features
       test_bot_connection('simulator') # Test enhanced simulator

    2. Test Enhanced Historical Data:
       test_historical_data('MRK', 'paper')      # Test real enhanced data
       test_historical_data('MRK', 'simulator')  # Test simulated enhanced data

    3. Run Enhanced Analysis (No Trading):
       results = run_enhanced_analysis_test(['MRK', 'UNH'])  # Comprehensive analysis

    4. Create Enhanced Trading Bot:
       bot = create_trading_bot('paper', 'enhanced')                           # Enhanced strategy
       bot = create_trading_bot('simulator', 'enhanced', short_window=5, long_window=20)
       bot = create_trading_bot('paper', 'enhanced', stop_loss_pct=0.03, take_profit_pct=0.10)

    5. Configure Enhanced Bot:
       bot.set_symbols(['MRK', 'UNH', 'LLY', 'PG'])        # Set symbols
       bot.update_interval = 30                             # Update every 30 seconds
       bot.strategy.stop_loss_pct = 0.05                    # 5% stop loss
       bot.strategy.take_profit_pct = 0.15                  # 15% take profit

    6. Run Enhanced Bot:
       bot.connect()                                        # Connect with enhanced features
       bot.start_trading(max_cycles=10)                     # Run with enhanced monitoring
       bot.disconnect()                                     # Clean disconnect

    7. Enhanced Backtest:
       config = {'short_window': 5, 'long_window': 15, 'stop_loss_pct': 0.03}
       results = run_strategy_backtest(['MRK', 'UNH'], duration_minutes=5, strategy_config=config)

    ENHANCED FEATURES:

    Technical Indicators:
    • RSI (Relative Strength Index) - Momentum oscillator
    • MACD (Moving Average Convergence Divergence) - Trend following
    • Bollinger Bands - Volatility and support/resistance
    • ATR (Average True Range) - Volatility measurement
    • Williams %R - Momentum indicator
    • Volume Analysis - Trading activity assessment
    • Support/Resistance Levels - Key price levels

    Advanced Signal Evaluation:
    • Multi-signal buy/sell decisions (7 different signal types)
    • Trend confirmation using daily data
    • Volume confirmation for signal strength
    • Momentum analysis across multiple timeframes

    Risk Management:
    • Automatic stop loss triggers (default 5%)
    • Take profit targets (default 15%)
    • Position sizing based on volatility
    • Maximum position size limits (default 20% of capital)
    • Real-time P&L monitoring

    Performance Tracking:
    • Win rate calculation
    • Realised and unrealised P&L
    • Trade history and signal logging
    • Comprehensive performance metrics
    • Portfolio value tracking

    Market Analysis:
    • Intraday data: 1-minute bars for up to 30 days
    • Daily data: Daily bars for months/years of history
    • Real-time market data integration
    • Enhanced caching to reduce API calls
    • Multiple symbol bulk analysis

    STRATEGY CONFIGURATION:

    Risk Parameters:
    bot.strategy.stop_loss_pct = 0.05      # 5% stop loss
    bot.strategy.take_profit_pct = 0.15    # 15% take profit  
    bot.strategy.max_position_size = 0.20  # 20% max position size

    Technical Parameters:
    short_window = 10    # Fast moving average period
    long_window = 30     # Slow moving average period

    IMPORTANT SAFETY NOTES:
    • Always test with 'simulator' first for strategy validation
    • Use 'paper' trading for realistic testing without risk
    • Enhanced risk management protects against large losses
    • Monitor technical indicators for market conditions
    • The enhanced strategy uses multiple confirmation signals
    • Real-time P&L tracking helps manage positions actively

    DIAGNOSTIC TOOLS:
    • bot.run_diagnostic_check() - Comprehensive system check
    • bot.get_portfolio_summary() - Detailed portfolio analysis
    • bot.strategy.get_market_overview(symbols) - Multi-symbol analysis
    • run_enhanced_analysis_test() - Strategy testing without trading

    ╔═══════════════════════════════════════════════════════════════════════════════════════╗
    """
    print(help_text)


def create_conservative_bot(environment: str = 'paper') -> IBTradingBot:
    """
    Create a conservative trading bot with enhanced risk management
    
    Args:
        environment: Trading environment
    """
    bot = create_trading_bot(
        environment=environment,
        strategy_type='enhanced',
        short_window=20,
        long_window=50,
        stop_loss_pct=0.03,      # Tight 3% stop loss
        take_profit_pct=0.08,    # Conservative 8% take profit
        max_position_size=0.10,  # Conservative 10% max position size
        name='Conservative_Enhanced'
    )
    
    # Set conservative symbols (large cap, stable stocks)
    bot.set_symbols(['MRK', 'UNH', 'PG', 'KO', 'WMT'])
    bot.update_interval = 60  # Conservative 1-minute updates
    
    return bot


def create_aggressive_bot(environment: str = 'paper') -> IBTradingBot:
    """
    Create an aggressive trading bot for higher risk/reward
    
    Args:
        environment: Trading environment
    """
    bot = create_trading_bot(
        environment=environment,
        strategy_type='enhanced',
        short_window=5,
        long_window=15,
        stop_loss_pct=0.08,      # Wider 8% stop loss
        take_profit_pct=0.25,    # Higher 25% take profit
        max_position_size=0.30,  # Larger 30% max position size
        name='Aggressive_Enhanced'
    )
    
    # Set growth/volatile symbols
    bot.set_symbols(['ABCL','ALLO','ANNX','ANRO','ANVS','APLM','APLT','APVO','ARAY','ARBE',
    'ATEN','ATAI','ATHA','AUTL','AVIR','CABA','CAMP','CARM','CATX','CCCC',
    'CELU','CERS','CGTX','CHRS','CLDI','CLLS','CLRB','CLSD','CLYM','CMPS',
    'CMPX','CMTL','COCP','CRBU','CRDF','CRDL','CRIS','CTMX','CTOR','CTXR',
    'CUE','CURR','DARE','DTIL','EDIT','ELDN','EQ','ERAS','ERNA','ESLA',
    'ESPR','EVGN','FATE','FBLG'])
    bot.update_interval = 30  # Faster 30-second updates
    
    return bot


def run_strategy_comparison(symbols: List[str] = None, duration_minutes: int = 10) -> Dict:
    """
    Compare conservative vs aggressive strategies
    
    Args:
        symbols: List of symbols to test
        duration_minutes: Test duration
    """
    print("Running strategy comparison test...")
    
    symbols = symbols or ['ABCL','ALLO','ANNX','ANRO','ANVS','APLM','APLT','APVO','ARAY','ARBE',
    'ATEN','ATAI','ATHA','AUTL','AVIR','CABA','CAMP','CARM','CATX','CCCC',
    'CELU','CERS','CGTX','CHRS','CLDI','CLLS','CLRB','CLSD','CLYM','CMPS',
    'CMPX','CMTL','COCP','CRBU','CRDF','CRDL','CRIS','CTMX','CTOR','CTXR',
    'CUE','CURR','DARE','DTIL','EDIT','ELDN','EQ','ERAS','ERNA','ESLA',
    'ESPR','EVGN','FATE','FBLG']
    
    # Conservative strategy configuration
    conservative_config = {
        'short_window': 20,
        'long_window': 50,
        'stop_loss_pct': 0.03,
        'take_profit_pct': 0.08,
        'name': 'Conservative_Test'
    }
    
    # Aggressive strategy configuration
    aggressive_config = {
        'short_window': 5,
        'long_window': 15,
        'stop_loss_pct': 0.08,
        'take_profit_pct': 0.25,
        'name': 'Aggressive_Test'
    }
    
    print(f"Testing {len(symbols)} symbols over {duration_minutes} minutes")
    print(f"Symbols: {symbols}")
    
    # Run both strategies
    print("\n--- Conservative Strategy ---")
    conservative_results = run_strategy_backtest(
        symbols, duration_minutes, conservative_config
    )
    
    print("\n--- Aggressive Strategy ---")
    aggressive_results = run_strategy_backtest(
        symbols, duration_minutes, aggressive_config
    )
    
    # Compare results
    if conservative_results and aggressive_results:
        print("\n" + "="*60)
        print("STRATEGY COMPARISON RESULTS")
        print("="*60)
        
        comparison = {
            'conservative': {
                'return_pct': conservative_results['return_percentage'],
                'trades': conservative_results['total_trades'],
                'win_rate': conservative_results['strategy_performance'].get('win_rate', 0),
                'final_value': conservative_results['portfolio_value']
            },
            'aggressive': {
                'return_pct': aggressive_results['return_percentage'],
                'trades': aggressive_results['total_trades'],
                'win_rate': aggressive_results['strategy_performance'].get('win_rate', 0),
                'final_value': aggressive_results['portfolio_value']
            }
        }
        
        print(f"Conservative Strategy:")
        print(f"  Return: {comparison['conservative']['return_pct']:+.2f}%")
        print(f"  Trades: {comparison['conservative']['trades']}")
        print(f"  Win Rate: {comparison['conservative']['win_rate']:.1f}%")
        print(f"  Final Value: £{comparison['conservative']['final_value']:.2f}")
        
        print(f"\nAggressive Strategy:")
        print(f"  Return: {comparison['aggressive']['return_pct']:+.2f}%")
        print(f"  Trades: {comparison['aggressive']['trades']}")
        print(f"  Win Rate: {comparison['aggressive']['win_rate']:.1f}%")
        print(f"  Final Value: £{comparison['aggressive']['final_value']:.2f}")
        
        # Determine winner
        conservative_score = comparison['conservative']['return_pct']
        aggressive_score = comparison['aggressive']['return_pct']
        
        if conservative_score > aggressive_score:
            winner = "Conservative"
            margin = conservative_score - aggressive_score
        elif aggressive_score > conservative_score:
            winner = "Aggressive"
            margin = aggressive_score - conservative_score
        else:
            winner = "Tie"
            margin = 0
        
        print(f"\nWinner: {winner} Strategy" + (f" (by {margin:.2f}%)" if margin > 0 else ""))
        print("="*60)
        
        return {
            'comparison': comparison,
            'winner': winner,
            'margin': margin,
            'conservative_results': conservative_results,
            'aggressive_results': aggressive_results
        }
    
    return {}


def run_multi_symbol_analysis(symbols: List[str] = None) -> Dict:
    """
    Run multi-symbol market analysis
    
    Args:
        symbols: List of symbols to analyse
    """
    symbols = symbols or ['ABCL','ALLO','ANNX','ANRO','ANVS','APLM','APLT','APVO','ARAY','ARBE',
    'ATEN','ATAI','ATHA','AUTL','AVIR','CABA','CAMP','CARM','CATX','CCCC',
    'CELU','CERS','CGTX','CHRS','CLDI','CLLS','CLRB','CLSD','CLYM','CMPS',
    'CMPX','CMTL','COCP','CRBU','CRDF','CRDL','CRIS','CTMX','CTOR','CTXR',
    'CUE','CURR','DARE','DTIL','EDIT','ELDN','EQ','ERAS','ERNA','ESLA',
    'ESPR','EVGN','FATE','FBLG']
    print(f"Running multi-symbol enhanced analysis for {len(symbols)} symbols...")
    
    bot = create_trading_bot('simulator', strategy_type='enhanced')
    
    if bot.connect():
        try:
            # Get market overview
            overview = bot.strategy.get_market_overview(symbols)
            
            print(f"\nMarket Overview:")
            print(f"Symbols Analysed: {len(overview['symbols_analysis'])}")
            print(f"Active Positions: {overview['total_positions']}")
            
            # Analyse each symbol
            market_conditions = {
                'bullish_signals': [],
                'bearish_signals': [],
                'neutral_signals': [],
                'high_volatility': [],
                'oversold': [],
                'overbought': []
            }
            
            for symbol in symbols:
                try:
                    cycle_result = bot.run_analysis_cycle(symbol)
                    
                    if 'error' not in cycle_result:
                        # Categorise market signals
                        buy_strength = cycle_result.get('buy_signal_count', 0)
                        sell_strength = cycle_result.get('sell_signal_count', 0)
                        buy_total = len(cycle_result.get('buy_signals', []))
                        sell_total = len(cycle_result.get('sell_signals', []))
                        
                        if buy_total > 0 and buy_strength >= buy_total // 2:
                            market_conditions['bullish_signals'].append(symbol)
                        elif sell_total > 0 and sell_strength >= sell_total // 2:
                            market_conditions['bearish_signals'].append(symbol)
                        else:
                            market_conditions['neutral_signals'].append(symbol)
                        
                        # Check technical conditions
                        indicators = cycle_result.get('technical_indicators', {})
                        if indicators:
                            if indicators.get('volatility', 0) > 3.0:
                                market_conditions['high_volatility'].append(symbol)
                            if indicators.get('rsi', 50) < 30:
                                market_conditions['oversold'].append(symbol)
                            elif indicators.get('rsi', 50) > 70:
                                market_conditions['overbought'].append(symbol)
                        
                        print(f"  {symbol}: £{cycle_result['current_price']:.2f} "
                              f"({cycle_result['price_change_pct']:+.2f}%) - "
                              f"Buy: {buy_strength}/{buy_total}, Sell: {sell_strength}/{sell_total}")
                        
                except Exception as e:
                    print(f"  {symbol}: Analysis error - {e}")
            
            # Display market conditions summary
            print(f"\nMarket Conditions Summary:")
            print(f"  Bullish Signals: {market_conditions['bullish_signals']}")
            print(f"  Bearish Signals: {market_conditions['bearish_signals']}")
            print(f"  Neutral/Hold: {market_conditions['neutral_signals']}")
            print(f"  High Volatility: {market_conditions['high_volatility']}")
            print(f"  Oversold (RSI < 30): {market_conditions['oversold']}")
            print(f"  Overbought (RSI > 70): {market_conditions['overbought']}")
            
            bot.disconnect()
            return market_conditions
            
        except Exception as e:
            print(f"Error in multi-symbol analysis: {e}")
            bot.disconnect()
            return {}
    else:
        print("Failed to connect to simulator")
        return {}


print("Enhanced Configuration and Testing Functions created")

print("\nAvailable Commands:")
print("• test_bot_connection('simulator')                    # Test enhanced simulator")
print("• test_historical_data('MRK', 'simulator')            # Test enhanced data analysis")
print("• run_enhanced_analysis_test(['MRK', 'UNH'])          # Analysis without trading")
print("• run_strategy_backtest(symbols, duration, config)    # Enhanced backtest")
print("• run_strategy_comparison(['MRK', 'UNH'], 10)         # Compare strategies")
print("• run_multi_symbol_analysis(['MRK', 'UNH', 'LLY'])    # Multi-symbol analysis")
print("• create_conservative_bot('simulator')                # Conservative strategy")
print("• create_aggressive_bot('simulator')                  # Aggressive strategy")

print("\nStrategy Types:")
print("• 'enhanced' - Advanced multi-indicator strategy (recommended)")
print("• 'sma'      - Simple moving average strategy (legacy)")

print("\nSafety Reminder: Always test with simulator and paper trading before live trading!")

Enhanced Configuration and Testing Functions created

Available Commands:
• test_bot_connection('simulator')                    # Test enhanced simulator
• test_historical_data('MRK', 'simulator')            # Test enhanced data analysis
• run_enhanced_analysis_test(['MRK', 'UNH'])          # Analysis without trading
• run_strategy_backtest(symbols, duration, config)    # Enhanced backtest
• run_strategy_comparison(['MRK', 'UNH'], 10)         # Compare strategies
• run_multi_symbol_analysis(['MRK', 'UNH', 'LLY'])    # Multi-symbol analysis
• create_conservative_bot('simulator')                # Conservative strategy
• create_aggressive_bot('simulator')                  # Aggressive strategy

Strategy Types:
• 'enhanced' - Advanced multi-indicator strategy (recommended)
• 'sma'      - Simple moving average strategy (legacy)

Safety Reminder: Always test with simulator and paper trading before live trading!


### Test Simulator Connection

In [28]:
print("="*60)
print("Enhanced Simulator Connection Test")
print("="*60)

# Test basic simulator connection
print("1. Testing Basic Simulator Connection:")
basic_connection_success = test_bot_connection('simulator')

if basic_connection_success:
    print("\n2. Testing Enhanced Analysis Features:")
    
    # Test enhanced analysis without trading
    analysis_results = run_enhanced_analysis_test(['ABCL','ALLO','ANNX','ANRO','ANVS','APLM','APLT','APVO','ARAY','ARBE',
    'ATEN','ATAI','ATHA','AUTL','AVIR','CABA','CAMP','CARM','CATX','CCCC',
    'CELU','CERS','CGTX','CHRS','CLDI','CLLS','CLRB','CLSD','CLYM','CMPS',
    'CMPX','CMTL','COCP','CRBU','CRDF','CRDL','CRIS','CTMX','CTOR','CTXR',
    'CUE','CURR','DARE','DTIL','EDIT','ELDN','EQ','ERAS','ERNA','ESLA',
    'ESPR','EVGN','FATE','FBLG'])
    
    if analysis_results:
        print(f"\nAnalysis completed for {len(analysis_results)} symbols")
        
        # Show summary of analysis results
        for symbol, result in analysis_results.items():
            if 'error' not in result:
                print(f"\n{symbol} Analysis Summary:")
                print(f"  Current Price: £{result['current_price']:.2f}")
                print(f"  Price Change: {result['price_change_pct']:+.2f}%")
                
                # Signal summary
                if result.get('buy_signals') and result.get('sell_signals'):
                    buy_count = result['buy_signal_count']
                    sell_count = result['sell_signal_count']
                    buy_total = len(result['buy_signals'])
                    sell_total = len(result['sell_signals'])
                    
                    signal_interpretation = "BUY" if buy_count >= buy_total//2 else \
                                          "SELL" if sell_count >= sell_total//2 else "HOLD"
                    
                    print(f"  Trading Signal: {signal_interpretation} (Buy: {buy_count}/{buy_total}, Sell: {sell_count}/{sell_total})")
                
                # Technical indicators summary
                if result.get('technical_indicators'):
                    indicators = result['technical_indicators']
                    tech_summary = []
                    
                    if 'rsi' in indicators:
                        rsi_level = "Oversold" if indicators['rsi'] < 30 else \
                                   "Overbought" if indicators['rsi'] > 70 else "Neutral"
                        tech_summary.append(f"RSI: {rsi_level}")
                    
                    if 'volatility' in indicators:
                        vol_level = "High" if indicators['volatility'] > 3.0 else \
                                   "Low" if indicators['volatility'] < 1.0 else "Normal"
                        tech_summary.append(f"Volatility: {vol_level}")
                    
                    if tech_summary:
                        print(f"  Market Conditions: {', '.join(tech_summary)}")
            else:
                print(f"\n{symbol}: Analysis Error - {result['error']}")

print("\n3. Testing Multi-Symbol Market Analysis:")

# Test multi-symbol analysis
market_conditions = run_multi_symbol_analysis(['MRK', 'UNH', 'LLY', 'PG'])

if market_conditions:
    print(f"\nMarket Sentiment Analysis:")
    total_symbols = len(market_conditions['bullish_signals']) + \
                   len(market_conditions['bearish_signals']) + \
                   len(market_conditions['neutral_signals'])
    
    if total_symbols > 0:
        bullish_pct = (len(market_conditions['bullish_signals']) / total_symbols) * 100
        bearish_pct = (len(market_conditions['bearish_signals']) / total_symbols) * 100
        neutral_pct = (len(market_conditions['neutral_signals']) / total_symbols) * 100
        
        print(f"  Bullish: {bullish_pct:.0f}% ({len(market_conditions['bullish_signals'])} symbols)")
        print(f"  Bearish: {bearish_pct:.0f}% ({len(market_conditions['bearish_signals'])} symbols)")
        print(f"  Neutral: {neutral_pct:.0f}% ({len(market_conditions['neutral_signals'])} symbols)")
        
        if market_conditions['high_volatility']:
            print(f"  High Volatility Warning: {market_conditions['high_volatility']}")

print("\n4. Testing Strategy Configuration:")

# Test different strategy configurations
print("\nTesting Conservative Strategy Configuration:")
conservative_bot = create_conservative_bot('simulator')
if conservative_bot.connect():
    print(f"  Strategy: {conservative_bot.strategy.name}")
    print(f"  Risk Management: Stop Loss {conservative_bot.strategy.stop_loss_pct*100:.1f}%, Take Profit {conservative_bot.strategy.take_profit_pct*100:.1f}%")
    print(f"  Position Sizing: Max {conservative_bot.strategy.max_position_size*100:.0f}% per position")
    print(f"  Symbols: {conservative_bot.symbols}")
    conservative_bot.disconnect()

print("\nTesting Aggressive Strategy Configuration:")
aggressive_bot = create_aggressive_bot('simulator')
if aggressive_bot.connect():
    print(f"  Strategy: {aggressive_bot.strategy.name}")
    print(f"  Risk Management: Stop Loss {aggressive_bot.strategy.stop_loss_pct*100:.1f}%, Take Profit {aggressive_bot.strategy.take_profit_pct*100:.1f}%")
    print(f"  Position Sizing: Max {aggressive_bot.strategy.max_position_size*100:.0f}% per position")
    print(f"  Symbols: {aggressive_bot.symbols}")
    aggressive_bot.disconnect()

print("\n5. Testing Custom Strategy Configuration:")

# Test custom strategy with specific parameters
custom_strategy_config = {
    'short_window': 8,
    'long_window': 21,
    'stop_loss_pct': 0.04,
    'take_profit_pct': 0.12,
    'max_position_size': 0.15,
    'name': 'Custom_Test_Strategy'
}

custom_bot = create_trading_bot(
    'simulator',
    strategy_type='enhanced',
    **custom_strategy_config
)

if custom_bot.connect():
    print(f"  Custom Strategy: {custom_bot.strategy.name}")
    print(f"  Moving Averages: {custom_bot.strategy.short_window}/{custom_bot.strategy.long_window}")
    print(f"  Risk Parameters: SL {custom_bot.strategy.stop_loss_pct*100:.1f}%, TP {custom_bot.strategy.take_profit_pct*100:.1f}%")
    
    # Test diagnostic check
    print(f"\n  Running Diagnostic Check:")
    diagnostics = custom_bot.run_diagnostic_check()
    
    if diagnostics:
        print(f"    Connection Status: {'Connected' if diagnostics['connection_status'] else 'Disconnected'}")
        
        components = diagnostics['components_initialised']
        initialised_count = sum(components.values())
        print(f"    Components Initialised: {initialised_count}/4")
        
        if diagnostics.get('data_availability'):
            available_data = sum(1 for symbol_data in diagnostics['data_availability'].values() 
                               if symbol_data.get('available', False))
            total_tested = len(diagnostics['data_availability'])
            print(f"    Data Availability: {available_data}/{total_tested} symbols")
    
    custom_bot.disconnect()

print("\n6. Testing Signal Evaluation System:")

# Create a bot specifically for signal testing
signal_test_bot = create_trading_bot('simulator', strategy_type='enhanced', short_window=5, long_window=15)

if signal_test_bot.connect():
    test_symbol = 'MRK'
    print(f"\nTesting signal evaluation for {test_symbol}:")
    
    try:
        # Get enhanced analysis
        analysis = signal_test_bot.get_enhanced_market_analysis(test_symbol)
        
        if analysis and analysis.get('technical_indicators'):
            indicators = analysis['technical_indicators']
            
            # Test buy signals
            buy_signals = signal_test_bot.strategy.evaluate_buy_signals(test_symbol, analysis, indicators)
            sell_signals = signal_test_bot.strategy.evaluate_sell_signals(test_symbol, analysis, indicators)
            
            print(f"  Current Price: £{analysis['last']:.2f}")
            print(f"  Buy Signal Types Active: {sum(buy_signals)}/{len(buy_signals)}")
            print(f"  Sell Signal Types Active: {sum(sell_signals)}/{len(sell_signals)}")
            
            # Show which specific signal types are active
            signal_names = [
                "SMA Crossover", "RSI Level", "Bollinger Position", 
                "MACD Trend", "Volume Confirmation", "Momentum", "Williams %R"
            ]
            
            active_buy_signals = [signal_names[i] for i, active in enumerate(buy_signals) if active]
            active_sell_signals = [signal_names[i] for i, active in enumerate(sell_signals) if active]
            
            if active_buy_signals:
                print(f"  Active Buy Signals: {', '.join(active_buy_signals)}")
            if active_sell_signals:
                print(f"  Active Sell Signals: {', '.join(active_sell_signals)}")
            
            # Test risk management
            risk_check = signal_test_bot.strategy.check_risk_management(test_symbol, analysis)
            print(f"  Risk Level: {risk_check.get('risk_level', 'Normal')}")
            
        else:
            print(f"  Enhanced analysis not available for {test_symbol}")
            
    except Exception as e:
        print(f"  Signal evaluation error: {e}")
    
    signal_test_bot.disconnect()

print("\n" + "="*60)
print("Enhanced Simulator Tests Completed")
print("="*60)
print("All enhanced features tested successfully!")
print("\nNext Steps:")
print("• Run enhanced backtest: run_strategy_backtest(['MRK', 'UNH'], 5)")
print("• Compare strategies: run_strategy_comparison(['MRK', 'UNH'], 5)")
print("• Test paper trading: test_bot_connection('paper')")
print("="*60)

2025-09-01 12:48:36,444 - INFO - Data handlers configured for enhanced analysis
2025-09-01 12:48:36,446 - INFO - Trading bot simulator initialised
2025-09-01 12:48:36,446 - INFO - Strategy: Enhanced_10_30
2025-09-01 12:48:36,447 - INFO - Starting balance: £10,000.00
2025-09-01 12:48:36,447 - INFO - Connected to trading simulator
2025-09-01 12:48:36,462 - INFO - Enhanced simulated analysis for MRK: £181.43 (+0.00%)
2025-09-01 12:48:36,463 - INFO - Disconnected from trading simulator
2025-09-01 12:48:36,464 - INFO - Data handlers configured for enhanced analysis
2025-09-01 12:48:36,464 - INFO - Trading bot simulator initialised
2025-09-01 12:48:36,464 - INFO - Strategy: Enhanced_10_30
2025-09-01 12:48:36,465 - INFO - Starting balance: £10,000.00
2025-09-01 12:48:36,465 - INFO - Connected to trading simulator
2025-09-01 12:48:36,476 - INFO - Enhanced simulated analysis for ABCL: £380.75 (+0.00%)
2025-09-01 12:48:36,483 - INFO - Enhanced simulated analysis for ALLO: £123.95 (+0.00%)
2025-0

Enhanced Simulator Connection Test
1. Testing Basic Simulator Connection:
Testing simulator trading connection...
Simulator connection test passed

Testing enhanced strategy features:
  MRK Indicators:
    RSI: 41.1
    MACD: 0.274
    Volatility: 0.2%
    Bollinger Band Width: 1.3%

2. Testing Enhanced Analysis Features:
Running enhanced analysis test for: ['ABCL', 'ALLO', 'ANNX', 'ANRO', 'ANVS', 'APLM', 'APLT', 'APVO', 'ARAY', 'ARBE', 'ATEN', 'ATAI', 'ATHA', 'AUTL', 'AVIR', 'CABA', 'CAMP', 'CARM', 'CATX', 'CCCC', 'CELU', 'CERS', 'CGTX', 'CHRS', 'CLDI', 'CLLS', 'CLRB', 'CLSD', 'CLYM', 'CMPS', 'CMPX', 'CMTL', 'COCP', 'CRBU', 'CRDF', 'CRDL', 'CRIS', 'CTMX', 'CTOR', 'CTXR', 'CUE', 'CURR', 'DARE', 'DTIL', 'EDIT', 'ELDN', 'EQ', 'ERAS', 'ERNA', 'ESLA', 'ESPR', 'EVGN', 'FATE', 'FBLG']

--- Analysing ABCL ---
Current Price: £380.75
Price Change: +0.00%
Position: 0 shares
Technical Analysis:
  RSI: 49.3 (Neutral)
  Volume Activity: 1.1x (Normal)
  volatility: 0.25% (Low)
Signal Analysis:
  Buy

2025-09-01 12:48:36,648 - INFO - Enhanced simulated analysis for CLSD: £214.04 (+0.00%)
2025-09-01 12:48:36,674 - INFO - Enhanced simulated analysis for CLYM: £367.16 (+0.00%)
2025-09-01 12:48:36,686 - INFO - Enhanced simulated analysis for CMPS: £59.83 (+0.00%)
2025-09-01 12:48:36,695 - INFO - Enhanced simulated analysis for CMPX: £495.00 (+0.00%)
2025-09-01 12:48:36,702 - INFO - Enhanced simulated analysis for CMTL: £211.22 (+0.00%)
2025-09-01 12:48:36,708 - INFO - Enhanced simulated analysis for COCP: £274.76 (+0.00%)
2025-09-01 12:48:36,715 - INFO - Enhanced simulated analysis for CRBU: £131.13 (+0.00%)
2025-09-01 12:48:36,721 - INFO - Enhanced simulated analysis for CRDF: £58.42 (+0.00%)
2025-09-01 12:48:36,728 - INFO - Enhanced simulated analysis for CRDL: £483.59 (+0.00%)
2025-09-01 12:48:36,734 - INFO - Enhanced simulated analysis for CRIS: £77.98 (+0.00%)
2025-09-01 12:48:36,740 - INFO - Enhanced simulated analysis for CTMX: £316.97 (+0.00%)
2025-09-01 12:48:36,746 - INFO - En

Current Price: £214.04
Price Change: +0.00%
Position: 0 shares
Technical Analysis:
  RSI: 72.0 (Overbought)
  Volume Activity: 0.8x (Normal)
  volatility: 0.26% (Low)
Signal Analysis:
  Buy Signals: 3/7
  Sell Signals: 2/7
  Overall Signal: BUY (Strength: 3/7)

--- Analysing CLYM ---
Current Price: £367.16
Price Change: +0.00%
Position: 0 shares
Technical Analysis:
  RSI: 63.9 (Neutral)
  Volume Activity: 0.6x (Low)
  volatility: 0.19% (Low)
Signal Analysis:
  Buy Signals: 2/7
  Sell Signals: 1/7
  Overall Signal: HOLD

--- Analysing CMPS ---
Current Price: £59.83
Price Change: +0.00%
Position: 0 shares
Technical Analysis:
  RSI: 49.0 (Neutral)
  Volume Activity: 0.8x (Normal)
  volatility: 0.19% (Low)
Signal Analysis:
  Buy Signals: 1/7
  Sell Signals: 1/7
  Overall Signal: HOLD

--- Analysing CMPX ---
Current Price: £495.00
Price Change: +0.00%
Position: 0 shares
Technical Analysis:
  RSI: 56.6 (Neutral)
  Volume Activity: 0.9x (Normal)
  volatility: 0.26% (Low)
Signal Analysis:
  Bu

2025-09-01 12:48:36,868 - INFO - Enhanced simulated analysis for PG: £159.53 (+0.00%)
2025-09-01 12:48:36,868 - INFO - Disconnected from trading simulator
2025-09-01 12:48:36,869 - INFO - Data handlers configured for enhanced analysis
2025-09-01 12:48:36,869 - INFO - Trading bot simulator initialised
2025-09-01 12:48:36,870 - INFO - Strategy: Conservative_Enhanced
2025-09-01 12:48:36,870 - INFO - Starting balance: £10,000.00
2025-09-01 12:48:36,870 - INFO - Updated trading symbols: ['MRK', 'UNH', 'PG', 'KO', 'WMT']
2025-09-01 12:48:36,871 - INFO - Connected to trading simulator
2025-09-01 12:48:36,871 - INFO - Disconnected from trading simulator
2025-09-01 12:48:36,871 - INFO - Data handlers configured for enhanced analysis
2025-09-01 12:48:36,872 - INFO - Trading bot simulator initialised
2025-09-01 12:48:36,872 - INFO - Strategy: Aggressive_Enhanced
2025-09-01 12:48:36,872 - INFO - Starting balance: £10,000.00
2025-09-01 12:48:36,873 - INFO - Updated trading symbols: ['ABCL', 'ALLO',

  PG: £159.53 (+0.00%) - Buy: 2/7, Sell: 1/7

Market Conditions Summary:
  Bullish Signals: []
  Bearish Signals: []
  Neutral/Hold: ['MRK', 'UNH', 'LLY', 'PG']
  High Volatility: []
  Oversold (RSI < 30): ['UNH']
  Overbought (RSI > 70): ['MRK', 'PG']

Market Sentiment Analysis:
  Bullish: 0% (0 symbols)
  Bearish: 0% (0 symbols)
  Neutral: 100% (4 symbols)

4. Testing Strategy Configuration:

Testing Conservative Strategy Configuration:
  Strategy: Conservative_Enhanced
  Risk Management: Stop Loss 3.0%, Take Profit 8.0%
  Position Sizing: Max 10% per position
  Symbols: ['MRK', 'UNH', 'PG', 'KO', 'WMT']

Testing Aggressive Strategy Configuration:
  Strategy: Aggressive_Enhanced
  Risk Management: Stop Loss 8.0%, Take Profit 25.0%
  Position Sizing: Max 30% per position
  Symbols: ['ABCL', 'ALLO', 'ANNX', 'ANRO', 'ANVS', 'APLM', 'APLT', 'APVO', 'ARAY', 'ARBE', 'ATEN', 'ATAI', 'ATHA', 'AUTL', 'AVIR', 'CABA', 'CAMP', 'CARM', 'CATX', 'CCCC', 'CELU', 'CERS', 'CGTX', 'CHRS', 'CLDI', 'CLLS

### Backtest Simulation

In [29]:
print("="*60)
print("Enhanced Backtest Simulation")
print("="*60)

# Simple enhanced backtest - 3 minutes on simulator
results = run_strategy_backtest(['ABCL','ALLO','ANNX','ANRO','ANVS','APLM','APLT','APVO','ARAY','ARBE',
    'ATEN','ATAI','ATHA','AUTL','AVIR','CABA','CAMP','CARM','CATX','CCCC',
    'CELU','CERS','CGTX','CHRS','CLDI','CLLS','CLRB','CLSD','CLYM','CMPS',
    'CMPX','CMTL','COCP','CRBU','CRDF','CRDL','CRIS','CTMX','CTOR','CTXR',
    'CUE','CURR','DARE','DTIL','EDIT','ELDN','EQ','ERAS','ERNA','ESLA',
    'ESPR','EVGN','FATE','FBLG'], duration_minutes=3)

if results:
    print(f"\nSimple Backtest Results:")
    print(f"Trades: {results['total_trades']}")
    print(f"Final Balance: £{results['final_balance']:,.2f}")
    print(f"Portfolio Value: £{results['portfolio_value']:,.2f}")
    print(f"Return: £{results['total_return']:+.2f} ({results['return_percentage']:+.2f}%)")
    
    # Show trades if any occurred
    if results['total_trades'] > 0:
        perf = results['strategy_performance']
        print(f"Win Rate: {perf['win_rate']:.1f}%")
        
        # Show positions
        for symbol, position in results['positions'].items():
            if position['shares'] > 0:
                print(f"{symbol}: {position['shares']} shares @ £{position['avg_price']:.2f}")
    else:
        print("No trades executed")
else:
    print("Backtest failed")

2025-09-01 12:48:39,294 - INFO - Data handlers configured for enhanced analysis
2025-09-01 12:48:39,295 - INFO - Trading bot simulator initialised
2025-09-01 12:48:39,296 - INFO - Strategy: Backtest_Enhanced
2025-09-01 12:48:39,297 - INFO - Starting balance: £10,000.00
2025-09-01 12:48:39,297 - INFO - Updated trading symbols: ['ABCL', 'ALLO', 'ANNX', 'ANRO', 'ANVS', 'APLM', 'APLT', 'APVO', 'ARAY', 'ARBE', 'ATEN', 'ATAI', 'ATHA', 'AUTL', 'AVIR', 'CABA', 'CAMP', 'CARM', 'CATX', 'CCCC', 'CELU', 'CERS', 'CGTX', 'CHRS', 'CLDI', 'CLLS', 'CLRB', 'CLSD', 'CLYM', 'CMPS', 'CMPX', 'CMTL', 'COCP', 'CRBU', 'CRDF', 'CRDL', 'CRIS', 'CTMX', 'CTOR', 'CTXR', 'CUE', 'CURR', 'DARE', 'DTIL', 'EDIT', 'ELDN', 'EQ', 'ERAS', 'ERNA', 'ESLA', 'ESPR', 'EVGN', 'FATE', 'FBLG']
2025-09-01 12:48:39,298 - INFO - Connected to trading simulator
2025-09-01 12:48:39,298 - INFO - Starting automated trading...
2025-09-01 12:48:39,299 - INFO - Symbols: ['ABCL', 'ALLO', 'ANNX', 'ANRO', 'ANVS', 'APLM', 'APLT', 'APVO', 'ARAY', 

Enhanced Backtest Simulation
Running enhanced strategy backtest simulation...
Running 36 cycles over 3 minutes...
Testing symbols: ['ABCL', 'ALLO', 'ANNX', 'ANRO', 'ANVS', 'APLM', 'APLT', 'APVO', 'ARAY', 'ARBE', 'ATEN', 'ATAI', 'ATHA', 'AUTL', 'AVIR', 'CABA', 'CAMP', 'CARM', 'CATX', 'CCCC', 'CELU', 'CERS', 'CGTX', 'CHRS', 'CLDI', 'CLLS', 'CLRB', 'CLSD', 'CLYM', 'CMPS', 'CMPX', 'CMTL', 'COCP', 'CRBU', 'CRDF', 'CRDL', 'CRIS', 'CTMX', 'CTOR', 'CTXR', 'CUE', 'CURR', 'DARE', 'DTIL', 'EDIT', 'ELDN', 'EQ', 'ERAS', 'ERNA', 'ESLA', 'ESPR', 'EVGN', 'FATE', 'FBLG']
Strategy: Backtest_Enhanced
Starting balance: £10,000.00
Risk Management: Stop Loss 5.0%, Take Profit 15.0%


2025-09-01 12:48:39,506 - INFO - Enhanced simulated analysis for CABA: £147.95 (+0.00%)
2025-09-01 12:48:39,518 - INFO - Enhanced simulated analysis for CAMP: £278.88 (+0.00%)
2025-09-01 12:48:39,529 - INFO - Enhanced simulated analysis for CARM: £483.60 (+0.00%)
2025-09-01 12:48:39,540 - INFO - Enhanced simulated analysis for CATX: £296.95 (+0.00%)
2025-09-01 12:48:39,552 - INFO - Enhanced simulated analysis for CCCC: £197.27 (+0.00%)
2025-09-01 12:48:39,563 - INFO - Enhanced simulated analysis for CELU: £428.48 (+0.00%)
2025-09-01 12:48:39,575 - INFO - Enhanced simulated analysis for CERS: £296.49 (+0.00%)
2025-09-01 12:48:39,586 - INFO - Enhanced simulated analysis for CGTX: £245.02 (+0.00%)
2025-09-01 12:48:39,593 - INFO - BUY signal for CGTX: 3/7 signals
2025-09-01 12:48:39,593 - INFO - Simulated BUY order placed: 4 CGTX
2025-09-01 12:48:39,593 - INFO - SIMULATED BUY: 4 CGTX @ £245.02 (Cost: £980.08, Balance: £8211.46)
2025-09-01 12:48:39,594 - INFO - CGTX indicators - RSI: 51.5, 


Enhanced Backtest Results:
Total Trades: 61
Cash Balance: £378.99
Portfolio Value: £9938.83
Total Return: £-61.17 (-0.61%)
Win Rate: 25.0%
Profitable Trades: 4/16
Realised P&L: £-78.28
Unrealised P&L: £+17.11

Final Positions:
  ANNX: 2 shares @ £404.23 (Cost: £808.46)
  CGTX: 4 shares @ £245.02 (Cost: £980.08)
  CHRS: 1 shares @ £146.10 (Cost: £146.10)
  CLSD: 5 shares @ £146.86 (Cost: £734.30)
  CMPS: 4 shares @ £194.19 (Cost: £776.76)
  CMTL: 2 shares @ £308.54 (Cost: £617.08)
  APLM: 1 shares @ £479.56 (Cost: £479.56)
  ARAY: 1 shares @ £388.64 (Cost: £388.64)
  CABA: 2 shares @ £147.96 (Cost: £295.92)
  EQ: 2 shares @ £167.34 (Cost: £334.68)
  APLT: 1 shares @ £388.13 (Cost: £388.13)
  ATEN: 1 shares @ £264.52 (Cost: £264.52)
  CLDI: 1 shares @ £191.49 (Cost: £191.49)
  CLRB: 1 shares @ £336.63 (Cost: £336.63)
  CTOR: 1 shares @ £148.00 (Cost: £148.00)
  FATE: 1 shares @ £150.89 (Cost: £150.89)
  CRDF: 1 shares @ £200.14 (Cost: £200.14)
  CRDL: 1 shares @ £113.26 (Cost: £113.26)


### Paper Trading Test (Requires IB connection)

In [31]:
print("="*60)
print("Paper Trading Test")
print("="*60)
print("NOTE: Requires TWS or IB Gateway running on port 7497")

# Simple paper trading connection test
success = test_bot_connection('paper')

if success:
    print("\nPaper trading connection successful")
    
    # Test one symbol with enhanced analysis
    print("\nTesting enhanced analysis on paper trading:")
    test_historical_data('MRK', 'paper')
    
else:
    print("\nPaper trading connection failed")
    print("\nSetup Instructions:")
    print("1. Start TWS or IB Gateway")
    print("2. Switch to Paper Trading mode")
    print("3. Configure API settings:")
    print("   - Enable 'ActiveX and Socket Clients'")
    print("   - Set Socket port to 7497")
    print("   - Disable 'Read-Only API'")
    print("   - Add 127.0.0.1 to trusted IPs")
    print("4. Restart TWS/Gateway")
    
    print("\nAlternative: Use simulator for testing")
    print("test_bot_connection('simulator')")

2025-09-01 12:52:50,721 - INFO - Initialised for Paper Trading (Demo Account)
2025-09-01 12:52:50,722 - INFO - Trading bot initialised for paper trading
2025-09-01 12:52:50,722 - INFO - Strategy: Enhanced_10_30


Paper Trading Test
NOTE: Requires TWS or IB Gateway running on port 7497
Testing paper trading connection...


2025-09-01 12:52:51,263 - INFO - ✓ Connected to IB on port 7497
2025-09-01 12:52:51,264 - INFO - Account: DUM797459
2025-09-01 12:52:51,265 - INFO - Data handlers configured for enhanced analysis
2025-09-01 12:52:51,266 - INFO - Trading bot connected successfully
2025-09-01 12:52:51,491 - INFO - Account Net Liquidation: £1000093.76
2025-09-01 12:52:51,491 - INFO - Buying Power: £4000000.00


Connection to paper trading successful

Testing enhanced analysis capabilities...


2025-09-01 12:52:53,017 - INFO - Retrieved 1170 historical bars for MRK
2025-09-01 12:52:53,384 - INFO - Retrieved 22 historical bars for MRK
2025-09-01 12:52:55,386 - WARNING - No valid market data for MRK
2025-09-01 12:52:55,389 - INFO - Enhanced analysis for MRK: £84.14 (+0.00%)


...

   Current price: £84.14
   Price change: +0.00%
   Technical Indicators:
     RSI: 71.2 (Overbought)
     MACD: 0.036 vs Signal: 0.012 (Bullish)
     Bollinger Bands: £83.83 - £84.16 (Within Bands)
     Volatility: 0.05% (Low)
     Volume Activity: 4.8x average (High)
   Signal Evaluation:
     Buy Signals: 3/7 active
     Sell Signals: 3/7 active
     Active Buy Signal Types: [0, 3, 4]
     Active Sell Signal Types: [1, 2, 5]
   Daily Trend: bullish (Strength: 1.3%)
   5-Day Momentum: -1.39%
   20-Day Momentum: +5.33%

2. Historical Data Timeframes:


2025-09-01 12:53:08,681 - INFO - Retrieved 390 historical bars for MRK


   Intraday (1 day): 390 data points
   Latest price: £84.14
   Date range: 2025-08-29 09:30:00-04:00 to 2025-08-29 15:59:00-04:00


2025-09-01 12:53:09,012 - INFO - Retrieved 22 historical bars for MRK
2025-09-01 12:53:09,014 - INFO - All market data subscriptions cancelled
2025-09-01 12:53:09,015 - INFO - Disconnected from IB
2025-09-01 12:53:09,016 - INFO - Trading bot disconnected


   Daily (1 month): 22 data points
   Monthly return: +7.68%

3. Strategy Status:
   Position: 0 shares
   Price History Length: 0
   Technical Indicators Count: 23
   Recent Signals: 0


### Advanced Trading Bot

In [32]:
print("="*60)
print("Advanced Trading Bot Test")
print("="*60)

# Create enhanced bot with custom settings
bot = create_trading_bot(
    environment='simulator',
    strategy_type='enhanced',
    short_window=5,
    long_window=15,
    stop_loss_pct=0.04,
    take_profit_pct=0.12
)

# Configure symbols and update speed
# Set the bot's tracked symbols
bot.set_symbols([
    'ABCL','ALLO','ANNX','ANRO','ANVS','APLM','APLT','APVO','ARAY','ARBE',
    'ATEN','ATAI','ATHA','AUTL','AVIR','CABA','CAMP','CARM','CATX','CCCC',
    'CELU','CERS','CGTX','CHRS','CLDI','CLLS','CLRB','CLSD','CLYM','CMPS',
    'CMPX','CMTL','COCP','CRBU','CRDF','CRDL','CRIS','CTMX','CTOR','CTXR',
    'CUE','CURR','DARE','DTIL','EDIT','ELDN','EQ','ERAS','ERNA','ESLA',
    'ESPR','EVGN','FATE','FBLG'
])
bot.update_interval = 3  # Fast updates for demo

print(f"Bot Environment: {bot.environment}")
print(f"Strategy: {bot.strategy.name}")
print(f"Symbols: {bot.symbols}")
print(f"Risk Management: SL {bot.strategy.stop_loss_pct*100:.1f}%, TP {bot.strategy.take_profit_pct*100:.1f}%")

# Connect and run
if bot.connect():
    print("\nRunning 5 cycles...")
    
    try:
        bot.start_trading(max_cycles=5)
        
        # Show results
        print(f"\nResults:")
        print(f"Trades: {bot.trades_executed}")
        print(f"Balance: £{bot.simulated_balance:,.2f}")
        
        # Show positions
        for symbol, position in bot.strategy.positions.items():
            if position['shares'] > 0:
                value = position['shares'] * position['avg_price']
                print(f"{symbol}: {position['shares']} shares @ £{position['avg_price']:.2f} (£{value:.2f})")
                
    finally:
        bot.disconnect()
else:
    print("Failed to connect")

2025-09-01 12:53:25,653 - INFO - Data handlers configured for enhanced analysis
2025-09-01 12:53:25,655 - INFO - Trading bot simulator initialised
2025-09-01 12:53:25,655 - INFO - Strategy: Enhanced_5_15
2025-09-01 12:53:25,658 - INFO - Starting balance: £10,000.00
2025-09-01 12:53:25,660 - INFO - Updated trading symbols: ['ABCL', 'ALLO', 'ANNX', 'ANRO', 'ANVS', 'APLM', 'APLT', 'APVO', 'ARAY', 'ARBE', 'ATEN', 'ATAI', 'ATHA', 'AUTL', 'AVIR', 'CABA', 'CAMP', 'CARM', 'CATX', 'CCCC', 'CELU', 'CERS', 'CGTX', 'CHRS', 'CLDI', 'CLLS', 'CLRB', 'CLSD', 'CLYM', 'CMPS', 'CMPX', 'CMTL', 'COCP', 'CRBU', 'CRDF', 'CRDL', 'CRIS', 'CTMX', 'CTOR', 'CTXR', 'CUE', 'CURR', 'DARE', 'DTIL', 'EDIT', 'ELDN', 'EQ', 'ERAS', 'ERNA', 'ESLA', 'ESPR', 'EVGN', 'FATE', 'FBLG']
2025-09-01 12:53:25,663 - INFO - Connected to trading simulator
2025-09-01 12:53:25,663 - INFO - Starting automated trading...
2025-09-01 12:53:25,664 - INFO - Symbols: ['ABCL', 'ALLO', 'ANNX', 'ANRO', 'ANVS', 'APLM', 'APLT', 'APVO', 'ARAY', 'ARB

Advanced Trading Bot Test
Bot Environment: simulator
Strategy: Enhanced_5_15
Symbols: ['ABCL', 'ALLO', 'ANNX', 'ANRO', 'ANVS', 'APLM', 'APLT', 'APVO', 'ARAY', 'ARBE', 'ATEN', 'ATAI', 'ATHA', 'AUTL', 'AVIR', 'CABA', 'CAMP', 'CARM', 'CATX', 'CCCC', 'CELU', 'CERS', 'CGTX', 'CHRS', 'CLDI', 'CLLS', 'CLRB', 'CLSD', 'CLYM', 'CMPS', 'CMPX', 'CMTL', 'COCP', 'CRBU', 'CRDF', 'CRDL', 'CRIS', 'CTMX', 'CTOR', 'CTXR', 'CUE', 'CURR', 'DARE', 'DTIL', 'EDIT', 'ELDN', 'EQ', 'ERAS', 'ERNA', 'ESLA', 'ESPR', 'EVGN', 'FATE', 'FBLG']
Risk Management: SL 4.0%, TP 12.0%

Running 5 cycles...


2025-09-01 12:53:25,861 - INFO - BUY signal for AUTL: 4/7 signals
2025-09-01 12:53:25,862 - INFO - Simulated BUY order placed: 1 AUTL
2025-09-01 12:53:25,862 - INFO - SIMULATED BUY: 1 AUTL @ £486.73 (Cost: £486.73, Balance: £7578.64)
2025-09-01 12:53:25,862 - INFO - AUTL indicators - RSI: 46.4, MACD: -0.440, Volatility: 0.17%
2025-09-01 12:53:25,869 - INFO - Enhanced simulated analysis for AVIR: £417.95 (+0.00%)
2025-09-01 12:53:25,882 - INFO - Enhanced simulated analysis for CABA: £185.60 (+0.00%)
2025-09-01 12:53:25,894 - INFO - Enhanced simulated analysis for CAMP: £147.28 (+0.00%)
2025-09-01 12:53:25,900 - INFO - BUY signal for CAMP: 3/7 signals
2025-09-01 12:53:25,900 - INFO - Simulated BUY order placed: 6 CAMP
2025-09-01 12:53:25,900 - INFO - SIMULATED BUY: 6 CAMP @ £147.28 (Cost: £883.68, Balance: £6694.96)
2025-09-01 12:53:25,901 - INFO - CAMP indicators - RSI: 73.8, MACD: 1.302, Volatility: 0.19%
2025-09-01 12:53:25,907 - INFO - Enhanced simulated analysis for CARM: £349.73 (+


Results:
Trades: 33
Balance: £60.73
ANNX: 5 shares @ £218.38 (£1091.90)
ATEN: 3 shares @ £280.91 (£842.73)
AUTL: 1 shares @ £485.08 (£485.08)
CAMP: 6 shares @ £147.28 (£883.68)
CARM: 2 shares @ £349.73 (£699.46)
CLRB: 1 shares @ £431.32 (£431.32)
CLYM: 3 shares @ £135.42 (£406.26)
CTMX: 5 shares @ £97.03 (£485.15)
CTOR: 5 shares @ £73.72 (£368.60)
ESLA: 1 shares @ £220.90 (£220.90)
ANVS: 4 shares @ £88.99 (£355.96)
CERS: 1 shares @ £495.61 (£495.61)
COCP: 1 shares @ £372.58 (£372.58)
ERNA: 3 shares @ £106.77 (£320.31)
FATE: 1 shares @ £449.23 (£449.23)
ABCL: 3 shares @ £101.56 (£304.68)
ANRO: 1 shares @ £271.31 (£271.31)
ATAI: 1 shares @ £355.80 (£355.80)
CABA: 1 shares @ £187.31 (£187.31)
DARE: 1 shares @ £139.95 (£139.95)
CLLS: 1 shares @ £125.61 (£125.61)
CRBU: 1 shares @ £327.36 (£327.36)
CRDF: 1 shares @ £55.46 (£55.46)
CTXR: 1 shares @ £209.45 (£209.45)
CGTX: 1 shares @ £58.68 (£58.68)


### Production-Ready Bot

In [33]:
def create_production_bot(environment: str = 'paper') -> IBTradingBot:
    """
    Create a production-ready trading bot with conservative settings
    
    Args:
        environment: 'paper', 'live', or 'simulator'
    """
    
    # Create bot with conservative enhanced strategy
    bot = create_trading_bot(
        environment=environment,
        strategy_type='enhanced',
        short_window=20,
        long_window=50,
        stop_loss_pct=0.05,
        take_profit_pct=0.15,
        name='Production_Enhanced'
    )
    
    # Set stable symbols
    bot.set_symbols(['ABCL','ALLO','ANNX','ANRO','ANVS','APLM','APLT','APVO','ARAY','ARBE',
    'ATEN','ATAI','ATHA','AUTL','AVIR','CABA','CAMP','CARM','CATX','CCCC',
    'CELU','CERS','CGTX','CHRS','CLDI','CLLS','CLRB','CLSD','CLYM','CMPS',
    'CMPX','CMTL','COCP','CRBU','CRDF','CRDL','CRIS','CTMX','CTOR','CTXR',
    'CUE','CURR','DARE','DTIL','EDIT','ELDN','EQ','ERAS','ERNA','ESLA',
    'ESPR','EVGN','FATE','FBLG'])
    bot.update_interval = 60  # 1-minute updates
    
    return bot


def run_production_session(environment: str = 'simulator', max_cycles: int = 10):
    """
    Run a simple production trading session
    
    Args:
        environment: 'simulator', 'paper', or 'live'
        max_cycles: Number of cycles to run
    """
    
    bot = create_production_bot(environment)
    
    try:
        if not bot.connect():
            print(f"Failed to connect to {environment}")
            return False
        
        print("="*50)
        print("PRODUCTION SESSION STARTED")
        print("="*50)
        print(f"Environment: {bot.environment.upper()}")
        print(f"Strategy: {bot.strategy.name}")
        print(f"Symbols: {bot.symbols}")
        print(f"Cycles: {max_cycles}")
        print("="*50)
        
        # Start trading
        bot.start_trading(max_cycles=max_cycles)
        
        # Show results
        print(f"\nSession Results:")
        print(f"Trades: {bot.trades_executed}")
        
        if environment == 'simulator':
            print(f"Final Balance: £{bot.simulated_balance:,.2f}")
        
        # Show positions
        any_positions = False
        for symbol, position in bot.strategy.positions.items():
            if position['shares'] > 0:
                print(f"{symbol}: {position['shares']} shares @ £{position['avg_price']:.2f}")
                any_positions = True
        
        if not any_positions:
            print("No positions")
        
        return True
        
    except KeyboardInterrupt:
        print("\nSession stopped by user")
        return True
    except Exception as e:
        print(f"Session error: {e}")
        return False
    finally:
        try:
            bot.disconnect()
        except:
            pass
        print("\nSession ended")


def run_quick_demo():
    """Run a quick 5-cycle demo"""
    print("Quick Demo (5 cycles, simulator)")
    success = run_production_session('simulator', max_cycles=5)
    
    if success:
        print("\nDemo completed")
        print("Next: Try run_production_session('paper', 10) with TWS running")
    else:
        print("Demo failed")


print("Production-ready bot functions created")
print("\nAvailable commands:")
print("• run_quick_demo()")
print("• run_production_session('simulator', 10)")
print("• run_production_session('paper', 10)")

print("\nWarning: Test extensively before live trading")

Production-ready bot functions created

Available commands:
• run_quick_demo()
• run_production_session('simulator', 10)
• run_production_session('paper', 10)



### Execute IB Trading Bot Simulator (Quick Demo)

In [34]:
print("="*60)
print("Quick Demo - Enhanced Trading Bot")
print("="*60)

# Simple 5-cycle demo
print("Running quick demo...")
run_quick_demo()

print("="*60)

2025-09-01 12:54:09,163 - INFO - Data handlers configured for enhanced analysis
2025-09-01 12:54:09,165 - INFO - Trading bot simulator initialised
2025-09-01 12:54:09,165 - INFO - Strategy: Production_Enhanced
2025-09-01 12:54:09,166 - INFO - Starting balance: £10,000.00
2025-09-01 12:54:09,166 - INFO - Updated trading symbols: ['ABCL', 'ALLO', 'ANNX', 'ANRO', 'ANVS', 'APLM', 'APLT', 'APVO', 'ARAY', 'ARBE', 'ATEN', 'ATAI', 'ATHA', 'AUTL', 'AVIR', 'CABA', 'CAMP', 'CARM', 'CATX', 'CCCC', 'CELU', 'CERS', 'CGTX', 'CHRS', 'CLDI', 'CLLS', 'CLRB', 'CLSD', 'CLYM', 'CMPS', 'CMPX', 'CMTL', 'COCP', 'CRBU', 'CRDF', 'CRDL', 'CRIS', 'CTMX', 'CTOR', 'CTXR', 'CUE', 'CURR', 'DARE', 'DTIL', 'EDIT', 'ELDN', 'EQ', 'ERAS', 'ERNA', 'ESLA', 'ESPR', 'EVGN', 'FATE', 'FBLG']
2025-09-01 12:54:09,167 - INFO - Connected to trading simulator
2025-09-01 12:54:09,167 - INFO - Starting automated trading...
2025-09-01 12:54:09,168 - INFO - Symbols: ['ABCL', 'ALLO', 'ANNX', 'ANRO', 'ANVS', 'APLM', 'APLT', 'APVO', 'ARAY'

Quick Demo - Enhanced Trading Bot
Running quick demo...
Quick Demo (5 cycles, simulator)
PRODUCTION SESSION STARTED
Environment: SIMULATOR
Strategy: Production_Enhanced
Symbols: ['ABCL', 'ALLO', 'ANNX', 'ANRO', 'ANVS', 'APLM', 'APLT', 'APVO', 'ARAY', 'ARBE', 'ATEN', 'ATAI', 'ATHA', 'AUTL', 'AVIR', 'CABA', 'CAMP', 'CARM', 'CATX', 'CCCC', 'CELU', 'CERS', 'CGTX', 'CHRS', 'CLDI', 'CLLS', 'CLRB', 'CLSD', 'CLYM', 'CMPS', 'CMPX', 'CMTL', 'COCP', 'CRBU', 'CRDF', 'CRDL', 'CRIS', 'CTMX', 'CTOR', 'CTXR', 'CUE', 'CURR', 'DARE', 'DTIL', 'EDIT', 'ELDN', 'EQ', 'ERAS', 'ERNA', 'ESLA', 'ESPR', 'EVGN', 'FATE', 'FBLG']
Cycles: 5


2025-09-01 12:54:09,369 - INFO - BUY signal for AUTL: 3/7 signals
2025-09-01 12:54:09,370 - INFO - Simulated BUY order placed: 1 AUTL
2025-09-01 12:54:09,370 - INFO - SIMULATED BUY: 1 AUTL @ £383.80 (Cost: £383.80, Balance: £5923.51)
2025-09-01 12:54:09,370 - INFO - AUTL indicators - RSI: 45.1, MACD: 0.772, Volatility: 0.18%
2025-09-01 12:54:09,379 - INFO - Enhanced simulated analysis for AVIR: £479.72 (+0.00%)
2025-09-01 12:54:09,391 - INFO - Enhanced simulated analysis for CABA: £150.25 (+0.00%)
2025-09-01 12:54:09,403 - INFO - Enhanced simulated analysis for CAMP: £55.65 (+0.00%)
2025-09-01 12:54:09,437 - INFO - Enhanced simulated analysis for CARM: £119.39 (+0.00%)
2025-09-01 12:54:09,453 - INFO - Enhanced simulated analysis for CATX: £436.41 (+0.00%)
2025-09-01 12:54:09,460 - INFO - BUY signal for CATX: 3/7 signals
2025-09-01 12:54:09,460 - INFO - Simulated BUY order placed: 1 CATX
2025-09-01 12:54:09,461 - INFO - SIMULATED BUY: 1 CATX @ £436.41 (Cost: £436.41, Balance: £5487.10)



Session Results:
Trades: 30
Final Balance: £147.10
ANNX: 3 shares @ £396.52
ANVS: 4 shares @ £258.48
APLT: 7 shares @ £106.91
AUTL: 1 shares @ £383.80
CATX: 1 shares @ £436.41
CLLS: 5 shares @ £126.63
ATHA: 5 shares @ £118.62
CCCC: 1 shares @ £459.74
CLDI: 1 shares @ £377.00
CUE: 1 shares @ £253.88
ANRO: 1 shares @ £210.05
CARM: 3 shares @ £118.67
CMTL: 1 shares @ £394.50
CRDF: 1 shares @ £343.56
CTMX: 1 shares @ £316.50
CTOR: 1 shares @ £435.02
EQ: 1 shares @ £347.26
ABCL: 1 shares @ £356.77
ALLO: 1 shares @ £464.62
CELU: 1 shares @ £102.36
CURR: 1 shares @ £70.11
EDIT: 1 shares @ £110.78
CAMP: 1 shares @ £56.22
CLSD: 1 shares @ £171.01

Session ended

Demo completed
Next: Try run_production_session('paper', 10) with TWS running


### Execute IB Trading Bot Simulator (Longer Session)

In [35]:
print("✓ IB Trading Bot Simulator Started")
run_production_session('simulator', max_cycles=20)
print("✓ IB Trading Bot Simulator Completed")

2025-09-01 13:03:07,600 - INFO - Data handlers configured for enhanced analysis
2025-09-01 13:03:07,603 - INFO - Trading bot simulator initialised
2025-09-01 13:03:07,603 - INFO - Strategy: Production_Enhanced
2025-09-01 13:03:07,604 - INFO - Starting balance: £10,000.00
2025-09-01 13:03:07,604 - INFO - Updated trading symbols: ['ABCL', 'ALLO', 'ANNX', 'ANRO', 'ANVS', 'APLM', 'APLT', 'APVO', 'ARAY', 'ARBE', 'ATEN', 'ATAI', 'ATHA', 'AUTL', 'AVIR', 'CABA', 'CAMP', 'CARM', 'CATX', 'CCCC', 'CELU', 'CERS', 'CGTX', 'CHRS', 'CLDI', 'CLLS', 'CLRB', 'CLSD', 'CLYM', 'CMPS', 'CMPX', 'CMTL', 'COCP', 'CRBU', 'CRDF', 'CRDL', 'CRIS', 'CTMX', 'CTOR', 'CTXR', 'CUE', 'CURR', 'DARE', 'DTIL', 'EDIT', 'ELDN', 'EQ', 'ERAS', 'ERNA', 'ESLA', 'ESPR', 'EVGN', 'FATE', 'FBLG']
2025-09-01 13:03:07,605 - INFO - Connected to trading simulator
2025-09-01 13:03:07,606 - INFO - Starting automated trading...
2025-09-01 13:03:07,606 - INFO - Symbols: ['ABCL', 'ALLO', 'ANNX', 'ANRO', 'ANVS', 'APLM', 'APLT', 'APVO', 'ARAY'

✓ IB Trading Bot Simulator Started
PRODUCTION SESSION STARTED
Environment: SIMULATOR
Strategy: Production_Enhanced
Symbols: ['ABCL', 'ALLO', 'ANNX', 'ANRO', 'ANVS', 'APLM', 'APLT', 'APVO', 'ARAY', 'ARBE', 'ATEN', 'ATAI', 'ATHA', 'AUTL', 'AVIR', 'CABA', 'CAMP', 'CARM', 'CATX', 'CCCC', 'CELU', 'CERS', 'CGTX', 'CHRS', 'CLDI', 'CLLS', 'CLRB', 'CLSD', 'CLYM', 'CMPS', 'CMPX', 'CMTL', 'COCP', 'CRBU', 'CRDF', 'CRDL', 'CRIS', 'CTMX', 'CTOR', 'CTXR', 'CUE', 'CURR', 'DARE', 'DTIL', 'EDIT', 'ELDN', 'EQ', 'ERAS', 'ERNA', 'ESLA', 'ESPR', 'EVGN', 'FATE', 'FBLG']
Cycles: 20


2025-09-01 13:03:07,813 - INFO - Enhanced simulated analysis for ATHA: £64.71 (+0.00%)
2025-09-01 13:03:07,826 - INFO - Enhanced simulated analysis for AUTL: £73.65 (+0.00%)
2025-09-01 13:03:07,839 - INFO - Enhanced simulated analysis for AVIR: £487.04 (+0.00%)
2025-09-01 13:03:07,851 - INFO - Enhanced simulated analysis for CABA: £147.29 (+0.00%)
2025-09-01 13:03:07,863 - INFO - Enhanced simulated analysis for CAMP: £493.99 (+0.00%)
2025-09-01 13:03:07,876 - INFO - Enhanced simulated analysis for CARM: £396.24 (+0.00%)
2025-09-01 13:03:07,887 - INFO - Enhanced simulated analysis for CATX: £378.90 (+0.00%)
2025-09-01 13:03:07,900 - INFO - Enhanced simulated analysis for CCCC: £119.43 (+0.00%)
2025-09-01 13:03:07,906 - INFO - BUY signal for CCCC: 3/7 signals
2025-09-01 13:03:07,906 - INFO - Simulated BUY order placed: 7 CCCC
2025-09-01 13:03:07,907 - INFO - SIMULATED BUY: 7 CCCC @ £119.43 (Cost: £836.01, Balance: £6505.33)
2025-09-01 13:03:07,907 - INFO - CCCC indicators - RSI: 32.0, MA


Session Results:
Trades: 62
Final Balance: £205.43
CLDI: 1 shares @ £148.68
CRDL: 2 shares @ £310.40
CTMX: 1 shares @ £480.38
CMTL: 1 shares @ £324.05
ANVS: 2 shares @ £286.36
CLRB: 1 shares @ £489.85
CMPS: 1 shares @ £202.62
CURR: 4 shares @ £105.39
APLT: 1 shares @ £460.17
CELU: 1 shares @ £459.36
CRBU: 1 shares @ £188.08
ERAS: 3 shares @ £83.18
CTOR: 1 shares @ £269.15
DTIL: 1 shares @ £427.57
ELDN: 1 shares @ £456.52
ESLA: 1 shares @ £361.24
ABCL: 1 shares @ £172.61
ARBE: 1 shares @ £309.40
ATEN: 1 shares @ £367.66
CABA: 1 shares @ £145.60
CGTX: 1 shares @ £286.59
CLYM: 1 shares @ £308.61
CHRS: 1 shares @ £59.04
CRDF: 1 shares @ £175.50
CLLS: 1 shares @ £82.42
CMPX: 1 shares @ £229.99
EDIT: 1 shares @ £459.05
ATHA: 1 shares @ £64.35
ANRO: 1 shares @ £292.95
CRIS: 1 shares @ £151.99
ESPR: 1 shares @ £93.06
CTXR: 1 shares @ £379.35

Session ended
✓ IB Trading Bot Simulator Completed


### Execution IB Trading Bot - Paper Trading (needs TWS running)

In [36]:
print("✓ IB Trading Bot Started")
run_production_session('paper', max_cycles=5)
print("✓ IB Trading Bot Completed")

2025-09-01 13:23:40,354 - INFO - Initialised for Paper Trading (Demo Account)
2025-09-01 13:23:40,355 - INFO - Trading bot initialised for paper trading
2025-09-01 13:23:40,356 - INFO - Strategy: Production_Enhanced
2025-09-01 13:23:40,357 - INFO - Updated trading symbols: ['ABCL', 'ALLO', 'ANNX', 'ANRO', 'ANVS', 'APLM', 'APLT', 'APVO', 'ARAY', 'ARBE', 'ATEN', 'ATAI', 'ATHA', 'AUTL', 'AVIR', 'CABA', 'CAMP', 'CARM', 'CATX', 'CCCC', 'CELU', 'CERS', 'CGTX', 'CHRS', 'CLDI', 'CLLS', 'CLRB', 'CLSD', 'CLYM', 'CMPS', 'CMPX', 'CMTL', 'COCP', 'CRBU', 'CRDF', 'CRDL', 'CRIS', 'CTMX', 'CTOR', 'CTXR', 'CUE', 'CURR', 'DARE', 'DTIL', 'EDIT', 'ELDN', 'EQ', 'ERAS', 'ERNA', 'ESLA', 'ESPR', 'EVGN', 'FATE', 'FBLG']


✓ IB Trading Bot Started


2025-09-01 13:23:40,875 - INFO - ✓ Connected to IB on port 7497
2025-09-01 13:23:40,875 - INFO - Account: DUM797459
2025-09-01 13:23:40,876 - INFO - Data handlers configured for enhanced analysis
2025-09-01 13:23:40,876 - INFO - Trading bot connected successfully
2025-09-01 13:23:41,101 - INFO - Account Net Liquidation: £1000093.76
2025-09-01 13:23:41,102 - INFO - Buying Power: £4000000.00
2025-09-01 13:23:41,103 - INFO - Starting automated trading...
2025-09-01 13:23:41,104 - INFO - Symbols: ['ABCL', 'ALLO', 'ANNX', 'ANRO', 'ANVS', 'APLM', 'APLT', 'APVO', 'ARAY', 'ARBE', 'ATEN', 'ATAI', 'ATHA', 'AUTL', 'AVIR', 'CABA', 'CAMP', 'CARM', 'CATX', 'CCCC', 'CELU', 'CERS', 'CGTX', 'CHRS', 'CLDI', 'CLLS', 'CLRB', 'CLSD', 'CLYM', 'CMPS', 'CMPX', 'CMTL', 'COCP', 'CRBU', 'CRDF', 'CRDL', 'CRIS', 'CTMX', 'CTOR', 'CTXR', 'CUE', 'CURR', 'DARE', 'DTIL', 'EDIT', 'ELDN', 'EQ', 'ERAS', 'ERNA', 'ESLA', 'ESPR', 'EVGN', 'FATE', 'FBLG']
2025-09-01 13:23:41,104 - INFO - Strategy: Production_Enhanced
2025-09-0

PRODUCTION SESSION STARTED
Environment: PAPER
Strategy: Production_Enhanced
Symbols: ['ABCL', 'ALLO', 'ANNX', 'ANRO', 'ANVS', 'APLM', 'APLT', 'APVO', 'ARAY', 'ARBE', 'ATEN', 'ATAI', 'ATHA', 'AUTL', 'AVIR', 'CABA', 'CAMP', 'CARM', 'CATX', 'CCCC', 'CELU', 'CERS', 'CGTX', 'CHRS', 'CLDI', 'CLLS', 'CLRB', 'CLSD', 'CLYM', 'CMPS', 'CMPX', 'CMTL', 'COCP', 'CRBU', 'CRDF', 'CRDL', 'CRIS', 'CTMX', 'CTOR', 'CTXR', 'CUE', 'CURR', 'DARE', 'DTIL', 'EDIT', 'ELDN', 'EQ', 'ERAS', 'ERNA', 'ESLA', 'ESPR', 'EVGN', 'FATE', 'FBLG']
Cycles: 5


2025-09-01 13:23:42,527 - INFO - Retrieved 1170 historical bars for ABCL
2025-09-01 13:23:42,866 - INFO - Retrieved 22 historical bars for ABCL
2025-09-01 13:23:44,870 - WARNING - No valid market data for ABCL
2025-09-01 13:23:44,872 - INFO - Enhanced analysis for ABCL: £4.17 (+0.00%)
2025-09-01 13:23:45,363 - INFO - Retrieved 1170 historical bars for ABCL
2025-09-01 13:23:45,699 - INFO - Retrieved 22 historical bars for ABCL
2025-09-01 13:23:47,703 - WARNING - No valid market data for ABCL
2025-09-01 13:23:48,208 - INFO - Retrieved 1170 historical bars for ALLO
2025-09-01 13:23:48,539 - INFO - Retrieved 22 historical bars for ALLO
2025-09-01 13:23:50,542 - WARNING - No valid market data for ALLO
2025-09-01 13:23:50,544 - INFO - Enhanced analysis for ALLO: £1.14 (+0.00%)
2025-09-01 13:23:51,033 - INFO - Retrieved 1170 historical bars for ALLO
2025-09-01 13:23:51,357 - INFO - Retrieved 22 historical bars for ALLO
2025-09-01 13:23:53,360 - WARNING - No valid market data for ALLO
2025-09-


Session Results:
Trades: 44
ALLO: 422907 shares @ £1.14
APLT: 958275 shares @ £0.50
ARAY: 314754 shares @ £1.52
ATEN: 27088 shares @ £17.72
ATAI: 105032 shares @ £4.57
CCCC: 184615 shares @ £2.60
CGTX: 198347 shares @ £2.42
CLRB: 96774 shares @ £4.96
CLYM: 213808 shares @ £2.25
CMPS: 108843 shares @ £4.41
CMPX: 137733 shares @ £3.48
CTOR: 223776 shares @ £2.15
CTXR: 361065 shares @ £1.33
ERAS: 304761 shares @ £1.57
ESLA: 457142 shares @ £1.05
FBLG: 670690 shares @ £0.60

Session ended
✓ IB Trading Bot Completed


### Live Trading Notes

Trader Workstation (TWS) Configuration for Live Trading:

1. Open TWS (not IB Gateway for first time)
2. Switch to LIVE account (not paper)
3. Go to Configure → API → Settings
4. Enable "Enable ActiveX and Socket Clients"
5. Set Socket port to 7496
6. Disable "Read-Only API"
7. Add 127.0.0.1 to trusted IPs
8. Restart TWS

**Remember**: Live trading = real money. Never rush into it. The simulator and paper trading are there for a reason! 🎯

### Execution IB Trading Bot - Live Trading (needs TWS running)

In [ ]:
# 💰 Recommended Live Trading Progression 💰

# Week 1: Extensive simulator testing
# run_production_session('simulator', max_cycles=100)

# Week 2: Extensive paper trading
# run_production_session('paper', max_cycles=100)

# Week 3: Micro live trading
run_production_session('live', max_cycles=2)     # Double cycle
# run_production_session('live', max_cycles=5)   # If successful

# Week 4+: Gradual scale-up
# run_production_session('live', max_cycles=10)  # If consistently profitable